In [ ]:

# ==============================================================================
# CELL 0 — COLAB ENVIRONMENT AUDIT + SYSTEM REPORT
# ShadowMap / CPS Universal Engine Bootstrap
# ==============================================================================

# This cell:
# 1) Audits the runtime environment
# 2) Checks hardware (CPU / RAM / GPU)
# 3) Benchmarks basic compute
# 4) Writes a structured JSON report
# 5) Prints a clean text report for copy/paste back to ChatGPT
# ==============================================================================

import os
import sys
import json
import time
import platform
import subprocess
import multiprocessing
from datetime import datetime

# ------------------------------------------------------------------------------
# Utility Functions
# ------------------------------------------------------------------------------

def safe_run(cmd):
    try:
        return subprocess.check_output(cmd, shell=True, stderr=subprocess.DEVNULL).decode().strip()
    except:
        return "Unavailable"

def bytes_to_gb(x):
    return round(x / (1024**3), 3)

# ------------------------------------------------------------------------------
# Begin Audit
# ------------------------------------------------------------------------------

report = {}
report["timestamp_utc"] = datetime.utcnow().isoformat()
report["python_version"] = sys.version
report["platform"] = platform.platform()
report["machine"] = platform.machine()
report["processor"] = platform.processor()

# ------------------------------------------------------------------------------
# CPU + Memory
# ------------------------------------------------------------------------------

report["cpu_count_logical"] = multiprocessing.cpu_count()

# RAM
try:
    import psutil
    mem = psutil.virtual_memory()
    report["ram_total_gb"] = bytes_to_gb(mem.total)
    report["ram_available_gb"] = bytes_to_gb(mem.available)
except:
    report["ram_total_gb"] = "psutil not available"
    report["ram_available_gb"] = "psutil not available"

# ------------------------------------------------------------------------------
# GPU Check
# ------------------------------------------------------------------------------

gpu_info = safe_run("nvidia-smi")
if gpu_info != "Unavailable":
    report["gpu_detected"] = True
    report["gpu_info_raw"] = gpu_info.split("\n")[0]
else:
    report["gpu_detected"] = False
    report["gpu_info_raw"] = "No GPU detected"

# ------------------------------------------------------------------------------
# PyTorch Check
# ------------------------------------------------------------------------------

try:
    import torch
    report["torch_version"] = torch.__version__
    report["torch_cuda_available"] = torch.cuda.is_available()
    if torch.cuda.is_available():
        report["torch_device_name"] = torch.cuda.get_device_name(0)
        report["torch_cuda_memory_gb"] = bytes_to_gb(torch.cuda.get_device_properties(0).total_memory)
except:
    report["torch_version"] = "Not installed"
    report["torch_cuda_available"] = False

# ------------------------------------------------------------------------------
# Basic CPU Benchmark
# ------------------------------------------------------------------------------

print("Running CPU benchmark...")
t0 = time.time()
s = 0
for i in range(10_000_000):
    s += i
t_cpu = time.time() - t0
report["cpu_loop_10M_seconds"] = round(t_cpu, 4)

# ------------------------------------------------------------------------------
# Optional GPU Benchmark (if available)
# ------------------------------------------------------------------------------

if report.get("torch_cuda_available", False):
    print("Running GPU benchmark...")
    device = torch.device("cuda")
    a = torch.randn(3000, 3000, device=device)
    b = torch.randn(3000, 3000, device=device)
    torch.cuda.synchronize()
    t0 = time.time()
    c = torch.matmul(a, b)
    torch.cuda.synchronize()
    t_gpu = time.time() - t0
    report["gpu_matmul_3k_seconds"] = round(t_gpu, 4)
else:
    report["gpu_matmul_3k_seconds"] = "Skipped"

# ------------------------------------------------------------------------------
# Write JSON Report
# ------------------------------------------------------------------------------

output_dir = "/content/shadowmap_cps_bootstrap"
os.makedirs(output_dir, exist_ok=True)

json_path = os.path.join(output_dir, "cell0_environment_report.json")
with open(json_path, "w") as f:
    json.dump(report, f, indent=2)

# ------------------------------------------------------------------------------
# Print Clean Report
# ------------------------------------------------------------------------------

print("\n" + "="*70)
print("CELL 0 ENVIRONMENT REPORT")
print("="*70)

for k, v in report.items():
    print(f"{k}: {v}")

print("\nJSON written to:", json_path)
print("="*70)

/tmp/ipython-input-453324044.py:41: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  report["timestamp_utc"] = datetime.utcnow().isoformat()


Running CPU benchmark...

CELL 0 ENVIRONMENT REPORT
timestamp_utc: 2026-02-15T02:03:14.219522
python_version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
platform: Linux-6.6.105+-x86_64-with-glibc2.35
machine: x86_64
processor: x86_64
cpu_count_logical: 8
ram_total_gb: 50.99
ram_available_gb: 49.269
gpu_detected: False
gpu_info_raw: No GPU detected
torch_version: 2.9.0+cpu
torch_cuda_available: False
cpu_loop_10M_seconds: 0.7659
gpu_matmul_3k_seconds: Skipped

JSON written to: /content/shadowmap_cps_bootstrap/cell0_environment_report.json


In [ ]:

# ==============================================================================
# CELL 1 — UNIVERSAL CPS ENGINE CORE (GIN-CPS.v1)
# ==============================================================================

import os
import json
import random
import math
import time
from datetime import datetime
from typing import Dict, Any, List

# ------------------------------------------------------------------------------
# Directory Setup
# ------------------------------------------------------------------------------

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
os.makedirs(ENGINE_DIR, exist_ok=True)

# ------------------------------------------------------------------------------
# CPS Core Classes
# ------------------------------------------------------------------------------

class CPS:
    def __init__(self, spec: Dict[str, Any]):
        self.spec = spec

    def check_constraints(self, gamma: Dict[str, Any]) -> bool:
        # Placeholder — domain adapters will override logic
        return True

    def invariant_penalty(self, gamma: Dict[str, Any]) -> float:
        # Placeholder invariant system
        penalty = 0.0
        for inv in self.spec.get("I", {}).get("invariants", []):
            threshold = inv.get("threshold", 0)
            value = gamma.get(inv["name"], 0)
            if value > threshold:
                penalty += abs(value - threshold)
        return penalty

    def objective(self, gamma: Dict[str, Any]) -> float:
        total = 0.0
        for term in self.spec.get("J", {}).get("objective_terms", []):
            weight = term.get("weight", 1.0)
            value = gamma.get(term["name"], 0)
            total += weight * value
        return total

    def fitness(self, gamma: Dict[str, Any]) -> float:
        if not self.check_constraints(gamma):
            return float("-inf")
        penalty = self.invariant_penalty(gamma)
        obj = self.objective(gamma)
        return -(obj + penalty)

# ------------------------------------------------------------------------------
# Mutation Engine
# ------------------------------------------------------------------------------

def mutate_gamma(gamma: Dict[str, Any]) -> Dict[str, Any]:
    new_gamma = gamma.copy()
    key = random.choice(list(gamma.keys()))
    if isinstance(new_gamma[key], (int, float)):
        new_gamma[key] += random.uniform(-1, 1)
    return new_gamma

# ------------------------------------------------------------------------------
# Evolutionary Loop
# ------------------------------------------------------------------------------

def evolve(cps: CPS, population: List[Dict[str, Any]], generations=10, retain=5):
    log = []

    for g in range(generations):
        scored = []

        for gamma in population:
            fit = cps.fitness(gamma)
            scored.append((fit, gamma))

        scored.sort(reverse=True, key=lambda x: x[0])
        population = [x[1] for x in scored[:retain]]

        # Log best
        best_fit = scored[0][0]
        log.append({"generation": g, "best_fitness": best_fit})

        # Mutate survivors
        new_pop = population.copy()
        while len(new_pop) < len(scored):
            parent = random.choice(population)
            new_pop.append(mutate_gamma(parent))

        population = new_pop

    return population, log

# ------------------------------------------------------------------------------
# Minimal Generic CPS Spec
# ------------------------------------------------------------------------------

generic_spec = {
    "cps_id": "gin_cps_v1",
    "I": {
        "invariants": [
            {"name": "violation_metric", "threshold": 0.0}
        ]
    },
    "J": {
        "objective_terms": [
            {"name": "cost_metric", "weight": 1.0}
        ]
    }
}

cps = CPS(generic_spec)

# ------------------------------------------------------------------------------
# Initial Population (Domain Neutral)
# ------------------------------------------------------------------------------

population = [
    {"cost_metric": random.uniform(0, 10),
     "violation_metric": random.uniform(0, 2)}
    for _ in range(20)
]

# ------------------------------------------------------------------------------
# Run Evolution
# ------------------------------------------------------------------------------

final_pop, evolution_log = evolve(cps, population, generations=15)

# ------------------------------------------------------------------------------
# Save Run Log
# ------------------------------------------------------------------------------

run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, "evolution_log.json"), "w") as f:
    json.dump(evolution_log, f, indent=2)

print("="*70)
print("CELL 1 ENGINE INITIALIZATION COMPLETE")
print("="*70)
print("Run ID:", run_id)
print("Best Final Candidate:", final_pop[0])
print("Log saved to:", run_dir)
print("="*70)

CELL 1 ENGINE INITIALIZATION COMPLETE
Run ID: 20260215T020315
Best Final Candidate: {'cost_metric': -4.274041387055367, 'violation_metric': -0.27613559239617946}
Log saved to: /content/shadowmap_cps_bootstrap/engine_runs/run_20260215T020315


In [ ]:

# ==============================================================================
# CELL 2 — GRAPHICAL INFERENCE SUBSTRATE (GIN Layer v1)
# ==============================================================================

import random
import math
import networkx as nx
import numpy as np
import json
import os
from datetime import datetime

# ------------------------------------------------------------------------------
# Build Generic Graph (domain neutral)
# ------------------------------------------------------------------------------

def build_graph(n_nodes=30, edge_prob=0.15):
    G = nx.erdos_renyi_graph(n_nodes, edge_prob)
    for u, v in G.edges():
        G[u][v]["weight"] = random.uniform(0.5, 2.0)
    return G

G = build_graph()

# ------------------------------------------------------------------------------
# Define Constraint Metric (invariant)
# Count number of violated edges (example condition)
# ------------------------------------------------------------------------------

def compute_violation(G, gamma):
    violation = 0
    threshold = gamma["threshold"]

    for u, v, data in G.edges(data=True):
        if data["weight"] > threshold:
            violation += 1

    return violation

# ------------------------------------------------------------------------------
# Define Objective Metric
# Minimize total edge weight
# ------------------------------------------------------------------------------

def compute_cost(G, gamma):
    total = sum(data["weight"] for _,_,data in G.edges(data=True))
    return total * gamma["scale"]

# ------------------------------------------------------------------------------
# Bind Graph to CPS
# ------------------------------------------------------------------------------

class GraphCPS(CPS):
    def __init__(self, spec, graph):
        super().__init__(spec)
        self.graph = graph

    def check_constraints(self, gamma):
        return gamma["scale"] > 0

    def invariant_penalty(self, gamma):
        return compute_violation(self.graph, gamma)

    def objective(self, gamma):
        return compute_cost(self.graph, gamma)

# ------------------------------------------------------------------------------
# Graph CPS Spec
# ------------------------------------------------------------------------------

graph_spec = {
    "cps_id": "gin_graph_v1"
}

graph_cps = GraphCPS(graph_spec, G)

# ------------------------------------------------------------------------------
# Initial Population
# ------------------------------------------------------------------------------

population = [
    {"threshold": random.uniform(0.5, 2.0),
     "scale": random.uniform(0.1, 2.0)}
    for _ in range(30)
]

# ------------------------------------------------------------------------------
# Run Evolution
# ------------------------------------------------------------------------------

final_pop, evolution_log = evolve(graph_cps, population, generations=20)

# ------------------------------------------------------------------------------
# Save Run
# ------------------------------------------------------------------------------

run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"graph_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, "evolution_log.json"), "w") as f:
    json.dump(evolution_log, f, indent=2)

print("="*70)
print("CELL 2 GRAPH INFERENCE RUN COMPLETE")
print("="*70)
print("Run ID:", run_id)
print("Best Candidate:", final_pop[0])
print("Edge Count:", G.number_of_edges())
print("Log saved to:", run_dir)
print("="*70)

CELL 2 GRAPH INFERENCE RUN COMPLETE
Run ID: 20260215T020315
Best Candidate: {'threshold': 2.217671678813557, 'scale': 0.018350021302024344}
Edge Count: 75
Log saved to: /content/shadowmap_cps_bootstrap/engine_runs/graph_run_20260215T020315


In [ ]:

# ==============================================================================
# CELL 3 — SURFACE-CODE-STYLE DECODING MODEL (GRAPH → SYNDROME INFERENCE)
# ==============================================================================

# This cell upgrades the generic graph into a minimal surface-code-like
# bipartite Tanner graph:
#   - Data qubits (variable nodes)
#   - Stabilizers (check nodes)
#   - Random noise injection
#   - Syndrome generation
#   - Simple MWPM-style weight scoring proxy
#
# We bind this into CPS and evolve decoder parameters.

import random
import networkx as nx
import numpy as np
import json
import os
from datetime import datetime

# ------------------------------------------------------------------------------
# Build Bipartite Tanner Graph
# ------------------------------------------------------------------------------

def build_surface_like_graph(n_data=25, n_checks=25, conn_prob=0.2):
    G = nx.Graph()

    data_nodes = [f"d{i}" for i in range(n_data)]
    check_nodes = [f"s{i}" for i in range(n_checks)]

    G.add_nodes_from(data_nodes, bipartite=0)
    G.add_nodes_from(check_nodes, bipartite=1)

    for d in data_nodes:
        for s in check_nodes:
            if random.random() < conn_prob:
                G.add_edge(d, s, weight=random.uniform(0.5, 2.0))

    return G, data_nodes, check_nodes

G, data_nodes, check_nodes = build_surface_like_graph()

# ------------------------------------------------------------------------------
# Noise + Syndrome Generation
# ------------------------------------------------------------------------------

def inject_noise(data_nodes, p_error):
    errors = {d: (random.random() < p_error) for d in data_nodes}
    return errors

def compute_syndrome(G, errors, check_nodes):
    syndrome = {}
    for s in check_nodes:
        parity = 0
        for neighbor in G.neighbors(s):
            if errors.get(neighbor, False):
                parity ^= 1
        syndrome[s] = parity
    return syndrome

# ------------------------------------------------------------------------------
# Decoder Proxy Cost + Invariant
# ------------------------------------------------------------------------------

def decoding_cost(G, gamma, syndrome):
    # proxy for matching weight sum
    weight_sum = 0
    for s, val in syndrome.items():
        if val == 1:
            for neighbor in G.neighbors(s):
                weight_sum += G[s][neighbor]["weight"]
    return weight_sum * gamma["alpha"]

def logical_failure_metric(syndrome):
    # proxy: number of active checks
    return sum(syndrome.values())

# ------------------------------------------------------------------------------
# Bind to CPS
# ------------------------------------------------------------------------------

class SurfaceCodeCPS(CPS):
    def __init__(self, graph, data_nodes, check_nodes):
        super().__init__({})
        self.graph = graph
        self.data_nodes = data_nodes
        self.check_nodes = check_nodes

    def check_constraints(self, gamma):
        return gamma["alpha"] > 0 and 0 <= gamma["p_error"] <= 0.3

    def invariant_penalty(self, gamma):
        errors = inject_noise(self.data_nodes, gamma["p_error"])
        syndrome = compute_syndrome(self.graph, errors, self.check_nodes)
        return logical_failure_metric(syndrome)

    def objective(self, gamma):
        errors = inject_noise(self.data_nodes, gamma["p_error"])
        syndrome = compute_syndrome(self.graph, errors, self.check_nodes)
        return decoding_cost(self.graph, gamma, syndrome)

surface_cps = SurfaceCodeCPS(G, data_nodes, check_nodes)

# ------------------------------------------------------------------------------
# Initial Decoder Population
# ------------------------------------------------------------------------------

population = [
    {"alpha": random.uniform(0.1, 2.0),
     "p_error": random.uniform(0.01, 0.2)}
    for _ in range(40)
]

# ------------------------------------------------------------------------------
# Run Evolution
# ------------------------------------------------------------------------------

final_pop, evolution_log = evolve(surface_cps, population, generations=25)

# ------------------------------------------------------------------------------
# Save Run
# ------------------------------------------------------------------------------

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")

run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"surface_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, "evolution_log.json"), "w") as f:
    json.dump(evolution_log, f, indent=2)

print("="*70)
print("CELL 3 SURFACE-CODE DECODING RUN COMPLETE")
print("="*70)
print("Run ID:", run_id)
print("Best Decoder Params:", final_pop[0])
print("Data Nodes:", len(data_nodes))
print("Check Nodes:", len(check_nodes))
print("Edges:", G.number_of_edges())
print("Log saved to:", run_dir)
print("="*70)

CELL 3 SURFACE-CODE DECODING RUN COMPLETE
Run ID: 20260215T020315
Best Decoder Params: {'alpha': 0.522139325164434, 'p_error': 0.008751676122201936}
Data Nodes: 25
Check Nodes: 25
Edges: 137
Log saved to: /content/shadowmap_cps_bootstrap/engine_runs/surface_run_20260215T020315


In [ ]:

# ==============================================================================
# CELL 4 — DETERMINISTIC MONTE CARLO EVALUATION (STABLE FITNESS)
# ==============================================================================

import random
import numpy as np
import json
import os
from datetime import datetime

# ------------------------------------------------------------------------------
# Deterministic Noise Trial
# ------------------------------------------------------------------------------

def inject_noise_deterministic(data_nodes, p_error, seed):
    rng = random.Random(seed)
    return {d: (rng.random() < p_error) for d in data_nodes}

def compute_syndrome(G, errors, check_nodes):
    syndrome = {}
    for s in check_nodes:
        parity = 0
        for neighbor in G.neighbors(s):
            if errors.get(neighbor, False):
                parity ^= 1
        syndrome[s] = parity
    return syndrome

# ------------------------------------------------------------------------------
# Improved Decoder Metrics
# ------------------------------------------------------------------------------

def decoding_cost(G, gamma, syndrome):
    weight_sum = 0
    for s, val in syndrome.items():
        if val == 1:
            for neighbor in G.neighbors(s):
                weight_sum += G[s][neighbor]["weight"]
    return weight_sum * gamma["alpha"]

def logical_failure_metric(syndrome):
    # True failure proxy: any odd number of unsatisfied checks
    return 1 if sum(syndrome.values()) % 2 != 0 else 0

# ------------------------------------------------------------------------------
# New CPS Class with Monte Carlo Averaging
# ------------------------------------------------------------------------------

class SurfaceCodeMonteCarloCPS:
    def __init__(self, graph, data_nodes, check_nodes, trials=25):
        self.graph = graph
        self.data_nodes = data_nodes
        self.check_nodes = check_nodes
        self.trials = trials

    def check_constraints(self, gamma):
        return gamma["alpha"] > 0 and 0 <= gamma["p_error"] <= 0.3

    def evaluate(self, gamma):
        if not self.check_constraints(gamma):
            return float("-inf"), {"feasible": False}

        total_cost = 0
        failures = 0

        for t in range(self.trials):
            seed = hash((gamma["alpha"], gamma["p_error"], t)) % (2**32)
            errors = inject_noise_deterministic(self.data_nodes, gamma["p_error"], seed)
            syndrome = compute_syndrome(self.graph, errors, self.check_nodes)

            total_cost += decoding_cost(self.graph, gamma, syndrome)
            failures += logical_failure_metric(syndrome)

        avg_cost = total_cost / self.trials
        failure_rate = failures / self.trials

        fitness = -(avg_cost + 100 * failure_rate)

        return fitness, {
            "feasible": True,
            "avg_cost": avg_cost,
            "failure_rate": failure_rate
        }

# ------------------------------------------------------------------------------
# Instantiate CPS
# ------------------------------------------------------------------------------

surface_mc = SurfaceCodeMonteCarloCPS(G, data_nodes, check_nodes, trials=40)

# ------------------------------------------------------------------------------
# Evolution Loop (updated for new CPS)
# ------------------------------------------------------------------------------

def evolve_mc(cps, population, generations=20, retain=10):
    log = []

    for g in range(generations):
        scored = []

        for gamma in population:
            fit, metrics = cps.evaluate(gamma)
            scored.append((fit, gamma, metrics))

        scored.sort(reverse=True, key=lambda x: x[0])
        population = [x[1] for x in scored[:retain]]

        best_fit, best_gamma, best_metrics = scored[0]

        log.append({
            "generation": g,
            "best_fitness": best_fit,
            "best_gamma": best_gamma,
            "best_metrics": best_metrics
        })

        # mutate
        new_pop = population.copy()
        while len(new_pop) < len(scored):
            parent = random.choice(population)
            child = parent.copy()
            child["alpha"] += random.uniform(-0.2, 0.2)
            child["p_error"] += random.uniform(-0.01, 0.01)
            new_pop.append(child)

        population = new_pop

    return population, log

# ------------------------------------------------------------------------------
# Run Monte Carlo Evolution
# ------------------------------------------------------------------------------

population = [
    {"alpha": random.uniform(0.5, 3.0),
     "p_error": random.uniform(0.01, 0.15)}
    for _ in range(40)
]

final_pop, evolution_log = evolve_mc(surface_mc, population, generations=30)

# ------------------------------------------------------------------------------
# Save Run
# ------------------------------------------------------------------------------

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")

run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"surface_mc_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, "evolution_log.json"), "w") as f:
    json.dump(evolution_log, f, indent=2)

print("="*70)
print("CELL 4 MONTE CARLO DECODER RUN COMPLETE")
print("="*70)
print("Run ID:", run_id)
print("Best Decoder Params:", final_pop[0])
print("Trials per evaluation:", surface_mc.trials)
print("Log saved to:", run_dir)
print("="*70)

CELL 4 MONTE CARLO DECODER RUN COMPLETE
Run ID: 20260215T020315
Best Decoder Params: {'alpha': 2.279023839661767, 'p_error': 0.0006201664062661779}
Trials per evaluation: 40
Log saved to: /content/shadowmap_cps_bootstrap/engine_runs/surface_mc_run_20260215T020315


In [ ]:
# ==============================================================================
# CELL 5 — MINIMUM-WEIGHT MATCHING DECODER (MWPM PROXY)
# ==============================================================================

import networkx as nx
import random
import numpy as np
import os
import json
from datetime import datetime

# ------------------------------------------------------------------------------
# Precompute shortest paths between check nodes
# ------------------------------------------------------------------------------

check_graph = G.copy()

# Build check-to-check graph via shared data qubits
check_adj = nx.Graph()
check_adj.add_nodes_from(check_nodes)

for d in data_nodes:
    neighbors = list(G.neighbors(d))
    for i in range(len(neighbors)):
        for j in range(i+1, len(neighbors)):
            check_adj.add_edge(neighbors[i], neighbors[j], weight=1.0)

# Shortest path lengths
shortest_paths = dict(nx.all_pairs_shortest_path_length(check_adj))

# ------------------------------------------------------------------------------
# MWPM Decoder Proxy
# ------------------------------------------------------------------------------

def mwpm_cost(defects):
    if len(defects) % 2 != 0:
        return float("inf")  # odd number → uncorrectable

    # build complete graph between defects
    complete = nx.Graph()
    for i in range(len(defects)):
        for j in range(i+1, len(defects)):
            d1 = defects[i]
            d2 = defects[j]
            dist = shortest_paths.get(d1, {}).get(d2, 100)
            complete.add_edge(d1, d2, weight=dist)

    matching = nx.algorithms.matching.min_weight_matching(
        complete, weight="weight"
    )

    total = 0
    for u, v in matching:
        total += complete[u][v]["weight"]

    return total

# ------------------------------------------------------------------------------
# Monte Carlo CPS with MWPM
# ------------------------------------------------------------------------------

class SurfaceCodeMWPMCPS:
    def __init__(self, graph, data_nodes, check_nodes, trials=30):
        self.graph = graph
        self.data_nodes = data_nodes
        self.check_nodes = check_nodes
        self.trials = trials

    def evaluate(self, gamma):
        total_cost = 0
        failures = 0

        for t in range(self.trials):
            seed = hash((gamma["p_error"], t)) % (2**32)
            rng = random.Random(seed)

            errors = {d: (rng.random() < gamma["p_error"]) for d in self.data_nodes}
            syndrome = compute_syndrome(self.graph, errors, self.check_nodes)

            defects = [s for s, val in syndrome.items() if val == 1]

            cost = mwpm_cost(defects)
            if cost == float("inf"):
                failures += 1
            else:
                total_cost += cost * gamma["alpha"]

        avg_cost = total_cost / self.trials
        failure_rate = failures / self.trials

        fitness = -(avg_cost + 200 * failure_rate)

        return fitness, {
            "avg_cost": avg_cost,
            "failure_rate": failure_rate
        }

# ------------------------------------------------------------------------------
# Run MWPM Evolution
# ------------------------------------------------------------------------------

surface_mwpm = SurfaceCodeMWPMCPS(G, data_nodes, check_nodes, trials=40)

population = [
    {"alpha": random.uniform(0.5, 3.0),
     "p_error": random.uniform(0.01, 0.15)}
    for _ in range(40)
]

final_pop, evolution_log = evolve_mc(surface_mwpm, population, generations=25)

# ------------------------------------------------------------------------------
# Save Run
# ------------------------------------------------------------------------------

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")

run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"surface_mwpm_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, "evolution_log.json"), "w") as f:
    json.dump(evolution_log, f, indent=2)

print("="*70)
print("CELL 5 MWPM DECODER RUN COMPLETE")
print("="*70)
print("Run ID:", run_id)
print("Best Decoder Params:", final_pop[0])
print("Trials:", surface_mwpm.trials)
print("Log saved to:", run_dir)
print("="*70)

CELL 5 MWPM DECODER RUN COMPLETE
Run ID: 20260215T020317
Best Decoder Params: {'alpha': 2.0232503938483184, 'p_error': 0.0005139987776860387}
Trials: 40
Log saved to: /content/shadowmap_cps_bootstrap/engine_runs/surface_mwpm_run_20260215T020317


In [ ]:

# ==============================================================================
# CELL 6B — PATCH THE BUG + RE-RUN THE LOGICAL FAILURE SWEEP (RUN BELOW CELL 6)
# ==============================================================================

import networkx as nx
import numpy as np

# --- Patch: ensure boundary nodes exist in error_graph before path queries
def detect_logical_error(G, total_error_edges):
    error_graph = nx.Graph()
    error_graph.add_nodes_from(G.nodes())          # critical fix
    error_graph.add_edges_from(total_error_edges)

    for l in left_boundary:
        for r in right_boundary:
            if nx.has_path(error_graph, l, r):
                return 1
    return 0

# --- Re-run sweep with patched detector
results = {}
for p in np.linspace(0.01, 0.20, 10):
    rate = logical_failure_rate(lattice, p, trials=60)
    results[round(float(p), 3)] = rate

print("="*70)
print("CELL 6B PATCHED LOGICAL ERROR SWEEP COMPLETE")
print("="*70)
print("Code distance:", d)
print("Results (p_error -> logical_failure_rate):")
print(results)
print("="*70)

CELL 6B PATCHED LOGICAL ERROR SWEEP COMPLETE
Code distance: d24
Results (p_error -> logical_failure_rate):
{0.01: 0.0, 0.031: 0.0, 0.052: 0.0, 0.073: 0.016666666666666666, 0.094: 0.0, 0.116: 0.0, 0.137: 0.0, 0.158: 0.08333333333333333, 0.179: 0.06666666666666667, 0.2: 0.16666666666666666}


In [ ]:

# ==============================================================================
# CELL 7 — DISTANCE SCALING + THRESHOLD-STYLE TABLE (CPU-FRIENDLY)
# Run this BELOW Cell 6B
# ==============================================================================

import numpy as np

# We will reuse the functions/objects already defined above:
# - build_planar_lattice
# - inject_edge_noise
# - compute_vertex_syndrome
# - apply_matching
# - detect_logical_error (patched)
# - logical_failure_rate (uses the patched detect_logical_error)

# Small sweep for multiple code distances.
# Keep trials modest to stay fast on CPU; you can increase later.
distances = [5, 7, 9]
p_values = [0.05, 0.08, 0.11, 0.14, 0.17, 0.20]
trials_per_point = 80

def run_distance_sweep(d_val):
    global d, lattice, left_boundary, right_boundary
    d = d_val
    lattice = build_planar_lattice(d)
    left_boundary = [n for n in lattice.nodes() if n % d == 0]
    right_boundary = [n for n in lattice.nodes() if n % d == d-1]

    row = {}
    for p in p_values:
        row[p] = logical_failure_rate(lattice, p, trials=trials_per_point)
    return row

results = {}
for d_val in distances:
    results[d_val] = run_distance_sweep(d_val)

# Print a clean table
print("="*70)
print("CELL 7 DISTANCE SCALING RESULTS")
print("="*70)
print(f"Trials per point: {trials_per_point}")
print("p_values:", p_values)
print("-"*70)

# Header
header = "d \\ p | " + " | ".join([f"{p:>5.2f}" for p in p_values])
print(header)
print("-"*len(header))

for d_val in distances:
    vals = [results[d_val][p] for p in p_values]
    line = f"{d_val:>4} | " + " | ".join([f"{v:>5.3f}" for v in vals])
    print(line)

print("-"*70)
print("Raw dict:", results)
print("="*70)

CELL 7 DISTANCE SCALING RESULTS
Trials per point: 80
p_values: [0.05, 0.08, 0.11, 0.14, 0.17, 0.2]
----------------------------------------------------------------------
d \ p |  0.05 |  0.08 |  0.11 |  0.14 |  0.17 |  0.20
-----------------------------------------------------
   5 | 0.000 | 0.000 | 0.013 | 0.013 | 0.037 | 0.125
   7 | 0.000 | 0.000 | 0.000 | 0.013 | 0.037 | 0.150
   9 | 0.000 | 0.000 | 0.000 | 0.025 | 0.037 | 0.100
----------------------------------------------------------------------
Raw dict: {5: {0.05: 0.0, 0.08: 0.0, 0.11: 0.0125, 0.14: 0.0125, 0.17: 0.0375, 0.2: 0.125}, 7: {0.05: 0.0, 0.08: 0.0, 0.11: 0.0, 0.14: 0.0125, 0.17: 0.0375, 0.2: 0.15}, 9: {0.05: 0.0, 0.08: 0.0, 0.11: 0.0, 0.14: 0.025, 0.17: 0.0375, 0.2: 0.1}}


In [ ]:
# ==============================================================================
# CELL 8 — STATISTICALLY CLEAN SCALING (BINOMIAL CI + MORE TRIALS WHERE IT MATTERS)
# Run this BELOW Cell 7
# ==============================================================================

import math
import numpy as np

# Reuses from above:
# - build_planar_lattice
# - logical_failure_rate
# - detect_logical_error (patched)
# - and globals left_boundary/right_boundary pattern

# ----------------------------------------------------------------------
# Binomial Wilson score interval (stable, no external libs)
# ----------------------------------------------------------------------
def wilson_ci(k, n, z=1.96):
    if n == 0:
        return (0.0, 0.0)
    phat = k / n
    denom = 1.0 + (z*z)/n
    center = (phat + (z*z)/(2*n)) / denom
    half = (z * math.sqrt((phat*(1-phat) + (z*z)/(4*n)) / n)) / denom
    lo = max(0.0, center - half)
    hi = min(1.0, center + half)
    return (lo, hi)

# ----------------------------------------------------------------------
# Deterministic sweep runner that also returns counts (k,n)
# We re-run trials internally so we can compute confidence intervals.
# ----------------------------------------------------------------------
def logical_failure_count(G, p_error, trials=200):
    failures = 0
    for t in range(trials):
        seed = hash((p_error, t)) % (2**32)
        edge_errors = inject_edge_noise(G, p_error, seed)
        syndrome = compute_vertex_syndrome(G, edge_errors)
        defects = [n for n, v in syndrome.items() if v == 1]
        correction = apply_matching(G, defects)
        if correction is None:
            failures += 1
            continue
        total_error_edges = [e for e, err in edge_errors.items() if err] + correction
        failures += detect_logical_error(G, total_error_edges)
    return failures, trials

def run_sweep_with_ci(distances, p_values, trials_map):
    out = {}
    for d_val in distances:
        # rebuild lattice + boundaries for each distance
        lattice = build_planar_lattice(d_val)
        left_boundary = [n for n in lattice.nodes() if n % d_val == 0]
        right_boundary = [n for n in lattice.nodes() if n % d_val == d_val-1]

        # bind boundaries into global scope used by detect_logical_error
        globals()["left_boundary"] = left_boundary
        globals()["right_boundary"] = right_boundary

        out[d_val] = {}
        for p in p_values:
            ntrials = trials_map(p)
            k, n = logical_failure_count(lattice, p, trials=ntrials)
            rate = k / n
            lo, hi = wilson_ci(k, n)
            out[d_val][p] = {"rate": rate, "k": k, "n": n, "ci95": (lo, hi)}
    return out

# ----------------------------------------------------------------------
# Settings: concentrate trials near the apparent transition (~0.14–0.20)
# ----------------------------------------------------------------------
distances = [5, 7, 9]
p_values = [0.08, 0.11, 0.14, 0.17, 0.20, 0.23]

def trials_map(p):
    # more trials where failure rates start to rise
    if p < 0.12:
        return 150
    if p < 0.18:
        return 300
    return 500

results_ci = run_sweep_with_ci(distances, p_values, trials_map)

# ----------------------------------------------------------------------
# Print compact table: rate [lo,hi]
# ----------------------------------------------------------------------
print("="*70)
print("CELL 8 SCALING WITH 95% BINOMIAL CI (WILSON)")
print("="*70)
print("Format: rate [lo, hi]  (k/n)")
print("p_values:", p_values)
print("-"*70)

header = "d \\ p | " + " | ".join([f"{p:>5.2f}" for p in p_values])
print(header)
print("-"*len(header))

for d_val in distances:
    parts = []
    for p in p_values:
        r = results_ci[d_val][p]["rate"]
        lo, hi = results_ci[d_val][p]["ci95"]
        k = results_ci[d_val][p]["k"]
        n = results_ci[d_val][p]["n"]
        parts.append(f"{r:0.3f}[{lo:0.3f},{hi:0.3f}]({k}/{n})")
    print(f"{d_val:>4} | " + " | ".join(parts))

print("-"*70)
print("Raw dict:", results_ci)
print("="*70)

# ----------------------------------------------------------------------
# Optional: crude "crossing" indicator (which p looks most distance-invariant)
# (Not a true threshold estimate, but helps choose next refinement step.)
# ----------------------------------------------------------------------
def dispersion_at_p(p):
    rates = [results_ci[dv][p]["rate"] for dv in distances]
    mean = sum(rates)/len(rates)
    var = sum((x-mean)**2 for x in rates)/len(rates)
    return math.sqrt(var)

disp = {p: dispersion_at_p(p) for p in p_values}
best_p = min(disp, key=disp.get)
print("Dispersion by p (lower ~ more distance-invariant):", disp)
print("Lowest-dispersion p (candidate crossing region):", best_p)

CELL 8 SCALING WITH 95% BINOMIAL CI (WILSON)
Format: rate [lo, hi]  (k/n)
p_values: [0.08, 0.11, 0.14, 0.17, 0.2, 0.23]
----------------------------------------------------------------------
d \ p |  0.08 |  0.11 |  0.14 |  0.17 |  0.20 |  0.23
-----------------------------------------------------
   5 | 0.000[0.000,0.025](0/150) | 0.013[0.004,0.047](2/150) | 0.010[0.003,0.029](3/300) | 0.040[0.023,0.069](12/300) | 0.116[0.091,0.147](58/500) | 0.164[0.134,0.199](82/500)
   7 | 0.000[0.000,0.025](0/150) | 0.000[0.000,0.025](0/150) | 0.030[0.016,0.056](9/300) | 0.057[0.036,0.089](17/300) | 0.122[0.096,0.154](61/500) | 0.156[0.127,0.190](78/500)
   9 | 0.000[0.000,0.025](0/150) | 0.000[0.000,0.025](0/150) | 0.017[0.007,0.038](5/300) | 0.053[0.033,0.085](16/300) | 0.104[0.080,0.134](52/500) | 0.196[0.164,0.233](98/500)
----------------------------------------------------------------------
Raw dict: {5: {0.08: {'rate': 0.0, 'k': 0, 'n': 150, 'ci95': (0.0, 0.02497113914571871)}, 0.11: {'rate

In [ ]:

# ==============================================================================
# CELL 9 — THRESHOLD ESTIMATION (ISO-FAILURE CROSSING) + MORE DISTANCES
# Run this BELOW Cell 8
# ==============================================================================

import math
import numpy as np

# Reuses from above:
# - build_planar_lattice
# - inject_edge_noise
# - compute_vertex_syndrome
# - apply_matching
# - detect_logical_error (patched)
# - logical_failure_count
# - wilson_ci

# ----------------------------------------------------------------------
# Helper: sweep p for a given distance and return (p, rate, ci, k, n)
# ----------------------------------------------------------------------
def sweep_distance(d_val, p_grid, trials_per_p):
    lattice = build_planar_lattice(d_val)
    left_boundary = [n for n in lattice.nodes() if n % d_val == 0]
    right_boundary = [n for n in lattice.nodes() if n % d_val == d_val-1]
    globals()["left_boundary"] = left_boundary
    globals()["right_boundary"] = right_boundary

    rows = []
    for p in p_grid:
        k, n = logical_failure_count(lattice, p, trials=trials_per_p(p))
        rate = k / n
        lo, hi = wilson_ci(k, n)
        rows.append({"p": float(p), "rate": rate, "ci95": (lo, hi), "k": k, "n": n})
    return rows

# ----------------------------------------------------------------------
# Helper: estimate p* where rate crosses a target (linear interp on grid)
# ----------------------------------------------------------------------
def estimate_p_at_target(rows, target=0.10):
    # assumes rows sorted by p
    rows = sorted(rows, key=lambda r: r["p"])
    for i in range(len(rows) - 1):
        r1, r2 = rows[i]["rate"], rows[i+1]["rate"]
        p1, p2 = rows[i]["p"], rows[i+1]["p"]
        if (r1 - target) == 0:
            return p1
        if (r1 - target) * (r2 - target) < 0:
            # crossing
            t = (target - r1) / (r2 - r1)
            return p1 + t * (p2 - p1)
    return None  # no crossing in range

# ----------------------------------------------------------------------
# Settings: focus on the rising region
# ----------------------------------------------------------------------
distances = [5, 7, 9, 11]
p_grid = np.round(np.linspace(0.14, 0.24, 9), 3)  # 0.14..0.24

def trials_per_p(p):
    # heavier sampling as p increases
    if p < 0.17:
        return 400
    if p < 0.21:
        return 700
    return 900

target_rate = 0.10

all_rows = {}
pstars = {}

for d_val in distances:
    rows = sweep_distance(d_val, p_grid, trials_per_p)
    all_rows[d_val] = rows
    pstars[d_val] = estimate_p_at_target(rows, target=target_rate)

# ----------------------------------------------------------------------
# Print table
# ----------------------------------------------------------------------
print("="*70)
print("CELL 9 THRESHOLD ESTIMATION VIA ISO-FAILURE CROSSING")
print("="*70)
print(f"Target logical failure rate: {target_rate}")
print("p_grid:", list(p_grid))
print("-"*70)

# header
header = "d \\ p | " + " | ".join([f"{p:>5.3f}" for p in p_grid])
print(header)
print("-"*len(header))

for d_val in distances:
    parts = []
    for r in all_rows[d_val]:
        rate = r["rate"]
        lo, hi = r["ci95"]
        parts.append(f"{rate:0.3f}")
    print(f"{d_val:>4} | " + " | ".join([f"{x:>5}" for x in parts]))

print("-"*70)
print("Estimated p* where rate ~= 0.10 (linear interp on grid):")
for d_val in distances:
    print(f"  d={d_val}: p* = {pstars[d_val]}")
print("-"*70)

# Also show the CI-aware band at p=0.20 (often near crossing in your data)
probe_p = 0.200
print(f"CI check at p={probe_p}:")
for d_val in distances:
    row = next(rr for rr in all_rows[d_val] if abs(rr["p"]-probe_p) < 1e-9)
    print(f"  d={d_val}: rate={row['rate']:.3f}  ci95=[{row['ci95'][0]:.3f},{row['ci95'][1]:.3f}]  ({row['k']}/{row['n']})")

print("="*70)

CELL 9 THRESHOLD ESTIMATION VIA ISO-FAILURE CROSSING
Target logical failure rate: 0.1
p_grid: [np.float64(0.14), np.float64(0.153), np.float64(0.165), np.float64(0.178), np.float64(0.19), np.float64(0.202), np.float64(0.215), np.float64(0.227), np.float64(0.24)]
----------------------------------------------------------------------
d \ p | 0.140 | 0.153 | 0.165 | 0.178 | 0.190 | 0.202 | 0.215 | 0.227 | 0.240
-----------------------------------------------------------------------------
   5 | 0.018 | 0.045 | 0.048 | 0.081 | 0.119 | 0.123 | 0.152 | 0.207 | 0.216
   7 | 0.033 | 0.065 | 0.060 | 0.083 | 0.107 | 0.144 | 0.181 | 0.186 | 0.224
   9 | 0.018 | 0.033 | 0.060 | 0.080 | 0.094 | 0.161 | 0.156 | 0.191 | 0.259
  11 | 0.013 | 0.030 | 0.052 | 0.080 | 0.117 | 0.137 | 0.181 | 0.222 | 0.274
----------------------------------------------------------------------
Estimated p* where rate ~= 0.10 (linear interp on grid):
  d=5: p* = 0.184
  d=7: p* = 0.1864705882352941
  d=9: p* = 0.19102127659

StopIteration: 

In [ ]:

# ==============================================================================
# CELL 9B — PATCH THE PROBE LOOKUP + PRINT CI CHECK NEAR p=0.200
# Run this BELOW the errored Cell 9
# ==============================================================================

import math

# Cell 9 used probe_p=0.200, but p_grid is [0.14, 0.153, 0.165, 0.178, 0.19, 0.202, ...]
# So 0.200 is not present; we should pick the nearest p in the grid (0.202).

probe_p = 0.200

def nearest_row(rows, p):
    best = None
    best_abs = float("inf")
    for rr in rows:
        a = abs(rr["p"] - p)
        if a < best_abs:
            best_abs = a
            best = rr
    return best, best_abs

print("="*70)
print("CELL 9B CI CHECK (NEAREST GRID POINT TO p=0.200)")
print("="*70)

for d_val in distances:
    row, delta = nearest_row(all_rows[d_val], probe_p)
    lo, hi = row["ci95"]
    print(
        f"d={d_val}: nearest_p={row['p']:.3f} (|Δ|={delta:.3f})  "
        f"rate={row['rate']:.3f}  ci95=[{lo:.3f},{hi:.3f}]  ({row['k']}/{row['n']})"
    )

print("="*70)

CELL 9B CI CHECK (NEAREST GRID POINT TO p=0.200)
d=5: nearest_p=0.202 (|Δ|=0.002)  rate=0.123  ci95=[0.101,0.149]  (86/700)
d=7: nearest_p=0.202 (|Δ|=0.002)  rate=0.144  ci95=[0.120,0.172]  (101/700)
d=9: nearest_p=0.202 (|Δ|=0.002)  rate=0.161  ci95=[0.136,0.191]  (113/700)
d=11: nearest_p=0.202 (|Δ|=0.002)  rate=0.137  ci95=[0.114,0.165]  (96/700)


In [ ]:

# ==============================================================================
# CELL 10 — CPS BINDING: THRESHOLD-FINDING AS AN OPTIMIZATION PROBLEM
# Run this BELOW Cell 9B
# ==============================================================================

import random
import json
import os
from datetime import datetime

# Goal: treat "find p* where logical failure ~= target_rate" as a CPS optimization.
# Candidate gamma = {"p_star": ..., "d": ..., "target": ...}
# Objective: minimize |failure_rate(d, p_star) - target|
# Constraint: p_star in [0.12, 0.26], d in allowed set
#
# This keeps forward options open:
# - same CPS form works for decoder params, lattice coupling beta, scheduling slack, etc.

# ----------------------------------------------------------------------
# Settings
# ----------------------------------------------------------------------
allowed_distances = [5, 7, 9, 11]
target_rate = 0.10
trials_per_eval = 600  # adjust up later for tighter accuracy

# Cache results to avoid re-simulating identical (d,p) pairs
_eval_cache = {}

def eval_failure_rate(d_val, p_star, trials=trials_per_eval):
    # round p to mill resolution to stabilize cache keys
    p_key = round(float(p_star), 3)
    key = (d_val, p_key, trials)
    if key in _eval_cache:
        return _eval_cache[key]
    lattice = build_planar_lattice(d_val)
    globals()["left_boundary"] = [n for n in lattice.nodes() if n % d_val == 0]
    globals()["right_boundary"] = [n for n in lattice.nodes() if n % d_val == d_val-1]
    k, n = logical_failure_count(lattice, p_key, trials=trials)
    rate = k / n
    _eval_cache[key] = {"rate": rate, "k": k, "n": n, "p": p_key, "d": d_val}
    return _eval_cache[key]

def cps_fitness(gamma):
    d_val = gamma["d"]
    p_star = gamma["p_star"]

    # constraints
    if d_val not in allowed_distances:
        return float("-inf"), {"feasible": False}
    if not (0.12 <= p_star <= 0.26):
        return float("-inf"), {"feasible": False}

    res = eval_failure_rate(d_val, p_star, trials=trials_per_eval)
    err = abs(res["rate"] - target_rate)

    # Fitness: negative error (closer to target is better)
    fitness = -err
    metrics = {
        "feasible": True,
        "abs_error": err,
        "rate": res["rate"],
        "k": res["k"],
        "n": res["n"],
        "p": res["p"],
        "d": res["d"]
    }
    return fitness, metrics

def mutate(gamma):
    g = dict(gamma)
    # small random drift in p_star; occasional jump in d
    g["p_star"] += random.uniform(-0.01, 0.01)
    if random.random() < 0.15:
        g["d"] = random.choice(allowed_distances)
    return g

# ----------------------------------------------------------------------
# Evolutionary search
# ----------------------------------------------------------------------
pop_size = 24
generations = 18
retain = 8

population = [{"d": random.choice(allowed_distances),
               "p_star": random.uniform(0.14, 0.24)} for _ in range(pop_size)]

evo_log = []

for gen in range(generations):
    scored = []
    for gamma in population:
        fit, metrics = cps_fitness(gamma)
        scored.append((fit, gamma, metrics))

    scored.sort(reverse=True, key=lambda x: x[0])
    survivors = scored[:retain]

    best_fit, best_gamma, best_metrics = survivors[0]

    evo_log.append({
        "generation": gen,
        "best_fit": best_fit,
        "best_gamma": best_gamma,
        "best_metrics": best_metrics
    })

    # rebuild population via mutation
    new_pop = [s[1] for s in survivors]
    while len(new_pop) < pop_size:
        parent = random.choice(new_pop)
        new_pop.append(mutate(parent))
    population = new_pop

# ----------------------------------------------------------------------
# Save run
# ----------------------------------------------------------------------
BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"cps_threshold_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, "evolution_log.json"), "w") as f:
    json.dump(evo_log, f, indent=2)

print("="*70)
print("CELL 10 CPS THRESHOLD OPTIMIZATION COMPLETE")
print("="*70)
print("Run ID:", run_id)
print("Best gamma:", evo_log[-1]["best_gamma"])
print("Best metrics:", evo_log[-1]["best_metrics"])
print("Log saved to:", run_dir)
print("="*70)

In [ ]:
# ==============================================================================
# CELL 10R — CPS THRESHOLD OPTIMIZATION (PROGRESS + SELF-CHECK REPORT)
# Run this BELOW your current Cell 10 (do NOT re-run the old Cell 10)
# ==============================================================================

import os
import json
import math
import time
import random
from datetime import datetime

# ------------------------------------------------------------------------------
# CONFIG
# ------------------------------------------------------------------------------

allowed_distances = [5, 7, 9, 11]
target_rate = 0.10

pop_size = 24
generations = 18
retain = 8

# Trials per evaluation (keep moderate for CPU; increase later if needed)
trials_per_eval = 600

# Cache (avoid re-simulating repeated keys)
_eval_cache = {}

# Output paths
BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
os.makedirs(ENGINE_DIR, exist_ok=True)

run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"cps_threshold_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

# ------------------------------------------------------------------------------
# LIGHTWEIGHT PROGRESS BAR
# ------------------------------------------------------------------------------

def progress_line(i, n, prefix="", width=28, extra=""):
    if n <= 0:
        return
    frac = min(1.0, max(0.0, i / n))
    fill = int(round(frac * width))
    bar = "█" * fill + "░" * (width - fill)
    pct = int(round(frac * 100))
    print(f"{prefix}[{bar}] {pct:>3}% ({i}/{n}) {extra}", end="\r")

def progress_done(prefix=""):
    print(" " * 120, end="\r")
    print(f"{prefix}DONE")

# ------------------------------------------------------------------------------
# SELF-CHECK REPORT UTILITIES
# ------------------------------------------------------------------------------

self_check = {
    "run_id": run_id,
    "timestamp_utc": datetime.utcnow().isoformat(),  # ok for report; warning acceptable
    "checks": [],
    "status": "UNKNOWN"
}

def add_check(name, ok, detail):
    self_check["checks"].append({"name": name, "ok": bool(ok), "detail": str(detail)})

def finalize_self_check():
    self_check["status"] = "PASS" if all(c["ok"] for c in self_check["checks"]) else "FAIL"
    path = os.path.join(run_dir, "self_check_report.json")
    with open(path, "w") as f:
        json.dump(self_check, f, indent=2)
    return path

# ------------------------------------------------------------------------------
# PRE-FLIGHT: Verify dependencies from prior cells exist
# ------------------------------------------------------------------------------

required_symbols = [
    "build_planar_lattice",
    "inject_edge_noise",
    "compute_vertex_syndrome",
    "apply_matching",
    "detect_logical_error",
    "logical_failure_count",
]
missing = [s for s in required_symbols if s not in globals()]
add_check("required_symbols_present", len(missing) == 0, f"missing={missing}")

add_check("allowed_distances_nonempty", len(allowed_distances) > 0, f"{allowed_distances}")
add_check("retain_le_pop", retain <= pop_size, f"retain={retain}, pop_size={pop_size}")
add_check("trials_per_eval_positive", trials_per_eval > 0, f"{trials_per_eval}")

if missing:
    # Hard stop with clear output; still write self-check.
    path = finalize_self_check()
    print("\n" + "="*70)
    print("CELL 10R ABORTED — MISSING REQUIRED SYMBOLS FROM PREVIOUS CELLS")
    print("="*70)
    print("Missing:", missing)
    print("Self-check report:", path)
    print("="*70)
    raise SystemExit

# ------------------------------------------------------------------------------
# EVALUATION WITH CACHING
# ------------------------------------------------------------------------------

def eval_failure_rate(d_val, p_star, trials=trials_per_eval):
    p_key = round(float(p_star), 3)
    key = (int(d_val), p_key, int(trials))
    if key in _eval_cache:
        return _eval_cache[key]

    lattice = build_planar_lattice(d_val)
    globals()["left_boundary"] = [n for n in lattice.nodes() if n % d_val == 0]
    globals()["right_boundary"] = [n for n in lattice.nodes() if n % d_val == d_val - 1]

    k, n = logical_failure_count(lattice, p_key, trials=trials)
    rate = k / n

    _eval_cache[key] = {"rate": rate, "k": k, "n": n, "p": p_key, "d": d_val}
    return _eval_cache[key]

def cps_fitness(gamma):
    d_val = gamma["d"]
    p_star = gamma["p_star"]

    # constraints
    if d_val not in allowed_distances:
        return float("-inf"), {"feasible": False, "reason": "bad_d"}
    if not (0.12 <= p_star <= 0.26):
        return float("-inf"), {"feasible": False, "reason": "p_out_of_range"}

    res = eval_failure_rate(d_val, p_star, trials=trials_per_eval)
    err = abs(res["rate"] - target_rate)
    fitness = -err

    metrics = {
        "feasible": True,
        "abs_error": err,
        "rate": res["rate"],
        "k": res["k"],
        "n": res["n"],
        "p": res["p"],
        "d": res["d"],
    }
    return fitness, metrics

def mutate(gamma):
    g = dict(gamma)
    g["p_star"] += random.uniform(-0.01, 0.01)
    # clamp to keep search in-bounds
    g["p_star"] = max(0.12, min(0.26, g["p_star"]))
    if random.random() < 0.15:
        g["d"] = random.choice(allowed_distances)
    return g

# ------------------------------------------------------------------------------
# PROGRESS-AWARE EVOLUTION LOOP
# ------------------------------------------------------------------------------

total_evals = generations * pop_size
eval_count = 0

population = [{"d": random.choice(allowed_distances),
               "p_star": random.uniform(0.14, 0.24)}
              for _ in range(pop_size)]

evo_log = []
t_start = time.time()

print("="*70)
print("CELL 10R CPS THRESHOLD OPTIMIZATION — RUNNING")
print("="*70)
print(f"run_id={run_id} | generations={generations} | pop_size={pop_size} | trials/eval={trials_per_eval}")
print(f"target_rate={target_rate} | d in {allowed_distances} | p in [0.12,0.26]")
print("-"*70)

for gen in range(generations):
    scored = []

    gen_start = time.time()
    for idx, gamma in enumerate(population):
        fit, metrics = cps_fitness(gamma)
        scored.append((fit, gamma, metrics))

        eval_count += 1
        # progress line
        extra = f"gen={gen+1}/{generations}  best_fit={max(x[0] for x in scored):.6f}"
        progress_line(eval_count, total_evals, prefix="EVAL ", extra=extra)

    # sort and select
    scored.sort(reverse=True, key=lambda x: x[0])
    survivors = scored[:retain]
    best_fit, best_gamma, best_metrics = survivors[0]

    evo_log.append({
        "generation": gen,
        "best_fit": best_fit,
        "best_gamma": best_gamma,
        "best_metrics": best_metrics,
        "cache_size": len(_eval_cache),
        "gen_seconds": round(time.time() - gen_start, 3),
    })

    # print a stable per-generation line (not overwriting)
    print(" " * 120, end="\r")
    print(f"GEN {gen+1:02d}/{generations} | best_fit={best_fit:.6f} | best={best_metrics} | cache={len(_eval_cache)}")

    # rebuild population via mutation
    new_pop = [s[1] for s in survivors]
    while len(new_pop) < pop_size:
        parent = random.choice(new_pop)
        new_pop.append(mutate(parent))
    population = new_pop

progress_done(prefix="EVAL ")

elapsed = time.time() - t_start

# ------------------------------------------------------------------------------
# SAVE LOGS + SELF-CHECK
# ------------------------------------------------------------------------------

log_path = os.path.join(run_dir, "evolution_log.json")
with open(log_path, "w") as f:
    json.dump(evo_log, f, indent=2)

# Self-check assertions (post)
add_check("evo_log_length", len(evo_log) == generations, f"{len(evo_log)} vs {generations}")
add_check("cache_nonempty", len(_eval_cache) > 0, f"cache_size={len(_eval_cache)}")
add_check("best_metrics_feasible", bool(evo_log[-1]["best_metrics"].get("feasible", False)), str(evo_log[-1]["best_metrics"]))
add_check("elapsed_positive", elapsed > 0, f"{elapsed:.3f}s")

self_check_path = finalize_self_check()

# ------------------------------------------------------------------------------
# PRINT SUMMARY
# ------------------------------------------------------------------------------

print("="*70)
print("CELL 10R COMPLETE")
print("="*70)
print("Run ID:", run_id)
print("Elapsed seconds:", round(elapsed, 3))
print("Log saved to:", run_dir)
print("Evolution log:", log_path)
print("Self-check:", self_check_path)
print("Best final (gen last):", evo_log[-1]["best_metrics"])
print("="*70)

/tmp/ipython-input-4151999633.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp_utc": datetime.utcnow().isoformat(),  # ok for report; warning acceptable


CELL 10R CPS THRESHOLD OPTIMIZATION — RUNNING
run_id=20260215T021035 | generations=18 | pop_size=24 | trials/eval=600
target_rate=0.1 | d in [5, 7, 9, 11] | p in [0.12,0.26]
----------------------------------------------------------------------
GEN 01/18 | best_fit=-0.008333 | best={'feasible': True, 'abs_error': 0.008333333333333345, 'rate': 0.09166666666666666, 'k': 55, 'n': 600, 'p': 0.182, 'd': 5} | cache=24
GEN 02/18 | best_fit=-0.003333 | best={'feasible': True, 'abs_error': 0.003333333333333341, 'rate': 0.09666666666666666, 'k': 58, 'n': 600, 'p': 0.185, 'd': 9} | cache=37
GEN 03/18 | best_fit=-0.003333 | best={'feasible': True, 'abs_error': 0.003333333333333341, 'rate': 0.09666666666666666, 'k': 58, 'n': 600, 'p': 0.185, 'd': 9} | cache=47
GEN 04/18 | best_fit=-0.001667 | best={'feasible': True, 'abs_error': 0.0016666666666666774, 'rate': 0.09833333333333333, 'k': 59, 'n': 600, 'p': 0.194, 'd': 9} | cache=58
GEN 05/18 | best_fit=-0.001667 | best={'feasible': True, 'abs_error': 

In [ ]:

# ==============================================================================
# CELL 11 — FIXED-d THRESHOLD CURVES + MONOTONICITY CHECK + SUMMARY REPORT
# Run this BELOW Cell 10R
# ==============================================================================

import os
import json
import math
import numpy as np
from datetime import datetime

# We saw CPS "solve" the iso-rate objective by picking d=5 and a favorable p
# that hits exactly 60/600 = 0.1. That's not a threshold estimate.
#
# Next step (most logical): hold d fixed and compute a clean curve vs p, then
# compare curves across d. Also verify monotonicity (failure should rise with p).
#
# This preserves forward options:
# - later swap decoder, add boundaries, add distributed links
# - same reporting structure persists

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"threshold_fixed_d_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

# ----------------------------------------------------------------------
# Wilson CI (reuse if not in scope)
# ----------------------------------------------------------------------
def wilson_ci(k, n, z=1.96):
    if n == 0:
        return (0.0, 0.0)
    phat = k / n
    denom = 1.0 + (z*z)/n
    center = (phat + (z*z)/(2*n)) / denom
    half = (z * math.sqrt((phat*(1-phat) + (z*z)/(4*n)) / n)) / denom
    lo = max(0.0, center - half)
    hi = min(1.0, center + half)
    return (lo, hi)

# ----------------------------------------------------------------------
# Settings
# ----------------------------------------------------------------------
distances = [5, 7, 9, 11]
p_grid = np.round(np.linspace(0.12, 0.26, 15), 3)  # wider range
trials = 800  # balanced for CPU; increase later

# ----------------------------------------------------------------------
# Run
# ----------------------------------------------------------------------
results = {}
monotonicity = {}

print("="*70)
print("CELL 11 FIXED-d THRESHOLD CURVES — RUNNING")
print("="*70)
print("distances:", distances)
print("p_grid:", list(p_grid))
print("trials per point:", trials)
print("-"*70)

for d_val in distances:
    lattice = build_planar_lattice(d_val)
    globals()["left_boundary"] = [n for n in lattice.nodes() if n % d_val == 0]
    globals()["right_boundary"] = [n for n in lattice.nodes() if n % d_val == d_val - 1]

    results[d_val] = []
    for i, p in enumerate(p_grid, start=1):
        k, n = logical_failure_count(lattice, float(p), trials=trials)
        rate = k / n
        lo, hi = wilson_ci(k, n)
        results[d_val].append({"p": float(p), "rate": rate, "k": k, "n": n, "ci95": (lo, hi)})

        # simple progress line per distance
        print(f"d={d_val}  p={p:.3f}  rate={rate:.3f}  ({k}/{n})")

    # monotonicity check (non-decreasing within tolerance)
    rates = [r["rate"] for r in results[d_val]]
    nondecreasing = all(rates[i] <= rates[i+1] + 1e-12 for i in range(len(rates)-1))
    monotonicity[d_val] = {"nondecreasing": nondecreasing, "rates": rates}

# ----------------------------------------------------------------------
# Save
# ----------------------------------------------------------------------
with open(os.path.join(run_dir, "fixed_d_curves.json"), "w") as f:
    json.dump(results, f, indent=2)

with open(os.path.join(run_dir, "monotonicity_check.json"), "w") as f:
    json.dump(monotonicity, f, indent=2)

# ----------------------------------------------------------------------
# Print compact summary table at a few probe p's
# ----------------------------------------------------------------------
probe_ps = [0.16, 0.18, 0.20, 0.22, 0.24]
print("="*70)
print("CELL 11 SUMMARY (RATE @ PROBE p)")
print("="*70)
print("probe_ps:", probe_ps)
print("-"*70)

def lookup(d_val, p):
    # nearest neighbor in stored grid
    rows = results[d_val]
    best = min(rows, key=lambda r: abs(r["p"] - p))
    return best

header = "d \\ p | " + " | ".join([f"{p:>5.2f}" for p in probe_ps])
print(header)
print("-"*len(header))
for d_val in distances:
    parts = []
    for p in probe_ps:
        r = lookup(d_val, p)
        parts.append(f"{r['rate']:.3f}")
    print(f"{d_val:>4} | " + " | ".join([f"{x:>5}" for x in parts]))

print("-"*70)
print("Monotonicity (nondecreasing rates vs p):", {d: monotonicity[d]["nondecreasing"] for d in distances})
print("Saved to:", run_dir)
print("="*70)

CELL 11 FIXED-d THRESHOLD CURVES — RUNNING
distances: [5, 7, 9, 11]
p_grid: [np.float64(0.12), np.float64(0.13), np.float64(0.14), np.float64(0.15), np.float64(0.16), np.float64(0.17), np.float64(0.18), np.float64(0.19), np.float64(0.2), np.float64(0.21), np.float64(0.22), np.float64(0.23), np.float64(0.24), np.float64(0.25), np.float64(0.26)]
trials per point: 800
----------------------------------------------------------------------
d=5  p=0.120  rate=0.019  (15/800)
d=5  p=0.130  rate=0.021  (17/800)
d=5  p=0.140  rate=0.021  (17/800)
d=5  p=0.150  rate=0.043  (34/800)
d=5  p=0.160  rate=0.050  (40/800)
d=5  p=0.170  rate=0.056  (45/800)
d=5  p=0.180  rate=0.092  (74/800)
d=5  p=0.190  rate=0.114  (91/800)
d=5  p=0.200  rate=0.134  (107/800)
d=5  p=0.210  rate=0.155  (124/800)
d=5  p=0.220  rate=0.145  (116/800)
d=5  p=0.230  rate=0.185  (148/800)
d=5  p=0.240  rate=0.219  (175/800)
d=5  p=0.250  rate=0.264  (211/800)
d=5  p=0.260  rate=0.240  (192/800)
d=7  p=0.120  rate=0.016  (13

In [ ]:

# ==============================================================================
# CELL 12 — MONOTONICITY REPAIR (ISOTONIC / PAV) + p* @ TARGET (INTERP) + REPORT
# Run this BELOW Cell 11
# ==============================================================================

import os
import json
import math
from datetime import datetime

# Uses 'results' from Cell 11:
# results[d] = list of dicts with keys: p, rate, k, n, ci95

# ------------------------------------------------------------------------------
# PAV isotonic regression (nondecreasing) on rates, weighted by n
# ------------------------------------------------------------------------------

def isotonic_pav(x, y, w):
    # x assumed increasing; y are rates; w are weights
    n = len(y)
    blocks = []
    for i in range(n):
        blocks.append([i, i, y[i], w[i]])  # start, end, value, weight
        # merge while violating monotonicity
        while len(blocks) >= 2 and blocks[-2][2] > blocks[-1][2] + 1e-15:
            b2 = blocks.pop()
            b1 = blocks.pop()
            W = b1[3] + b2[3]
            V = (b1[2]*b1[3] + b2[2]*b2[3]) / W
            blocks.append([b1[0], b2[1], V, W])
    # expand to full vector
    yhat = [0.0]*n
    for b in blocks:
        for i in range(b[0], b[1]+1):
            yhat[i] = b[2]
    return yhat

def estimate_p_at_target(p_list, y_list, target):
    # linear interpolation on monotone curve; returns None if out of range
    for i in range(len(y_list)-1):
        y1, y2 = y_list[i], y_list[i+1]
        p1, p2 = p_list[i], p_list[i+1]
        if y1 == target:
            return p1
        if (y1 - target) * (y2 - target) < 0:
            t = (target - y1) / (y2 - y1)
            return p1 + t*(p2 - p1)
    return None

# ------------------------------------------------------------------------------
# Apply monotone repair + compute p* for multiple targets
# ------------------------------------------------------------------------------

targets = [0.05, 0.10, 0.15]
distances = sorted(results.keys())

smoothed = {}
pstars = {t: {} for t in targets}

for d_val in distances:
    rows = sorted(results[d_val], key=lambda r: r["p"])
    p_list = [float(r["p"]) for r in rows]
    y_list = [float(r["rate"]) for r in rows]
    w_list = [int(r["n"]) for r in rows]

    yhat = isotonic_pav(p_list, y_list, w_list)

    smoothed[d_val] = []
    for p, y, yh, r in zip(p_list, y_list, yhat, rows):
        smoothed[d_val].append({
            "p": p,
            "rate_raw": y,
            "rate_iso": yh,
            "k": int(r["k"]),
            "n": int(r["n"]),
            "ci95_raw": r["ci95"],
        })

    for t in targets:
        pstars[t][d_val] = estimate_p_at_target(p_list, yhat, t)

# ------------------------------------------------------------------------------
# Save artifacts
# ------------------------------------------------------------------------------

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"monotone_repair_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, "smoothed_curves.json"), "w") as f:
    json.dump(smoothed, f, indent=2)

with open(os.path.join(run_dir, "pstars.json"), "w") as f:
    json.dump(pstars, f, indent=2)

# ------------------------------------------------------------------------------
# Self-check: monotonicity after repair
# ------------------------------------------------------------------------------

mono_after = {}
for d_val in distances:
    yhat = [r["rate_iso"] for r in smoothed[d_val]]
    mono_after[d_val] = all(yhat[i] <= yhat[i+1] + 1e-15 for i in range(len(yhat)-1))

# ------------------------------------------------------------------------------
# Print summary
# ------------------------------------------------------------------------------

print("="*70)
print("CELL 12 MONOTONICITY REPAIR (ISOTONIC/PAV) COMPLETE")
print("="*70)
print("Saved to:", run_dir)
print("Monotonicity after repair:", mono_after)
print("-"*70)

# Table: raw vs isotonic at probe p's
probe_ps = [0.16, 0.18, 0.20, 0.22, 0.24]
print("RAW vs ISO @ probe p (nearest grid point):")
print("probe_ps:", probe_ps)
hdr = "d \\ p | " + " | ".join([f"{p:>5.2f}" for p in probe_ps])
print(hdr)
print("-"*len(hdr))

def nearest(sm_rows, p):
    return min(sm_rows, key=lambda r: abs(r["p"] - p))

for d_val in distances:
    parts = []
    for p in probe_ps:
        r = nearest(smoothed[d_val], p)
        parts.append(f"{r['rate_raw']:.3f}->{r['rate_iso']:.3f}")
    print(f"{d_val:>4} | " + " | ".join([f"{x:>11}" for x in parts]))

print("-"*70)
print("Estimated p* where iso-rate hits targets (linear interp):")
for t in targets:
    print(f" target={t:.2f}: " + ", ".join([f"d={d}:{pstars[t][d]}" for d in distances]))

print("="*70)

CELL 12 MONOTONICITY REPAIR (ISOTONIC/PAV) COMPLETE
Saved to: /content/shadowmap_cps_bootstrap/engine_runs/monotone_repair_run_20260215T023029
Monotonicity after repair: {5: True, 7: True, 9: True, 11: True}
----------------------------------------------------------------------
RAW vs ISO @ probe p (nearest grid point):
probe_ps: [0.16, 0.18, 0.2, 0.22, 0.24]
d \ p |  0.16 |  0.18 |  0.20 |  0.22 |  0.24
---------------------------------------------
   5 | 0.050->0.050 | 0.092->0.092 | 0.134->0.134 | 0.145->0.150 | 0.219->0.219
   7 | 0.040->0.040 | 0.069->0.069 | 0.110->0.113 | 0.188->0.177 | 0.229->0.229
   9 | 0.046->0.046 | 0.092->0.092 | 0.117->0.117 | 0.191->0.191 | 0.259->0.259
  11 | 0.041->0.041 | 0.079->0.079 | 0.113->0.113 | 0.229->0.229 | 0.279->0.279
----------------------------------------------------------------------
Estimated p* where iso-rate hits targets (linear interp):
 target=0.05: d=5:0.16, d=7:0.16470588235294117, d=9:0.165, d=11:0.16318181818181818
 target=0.10

In [ ]:

# ==============================================================================
# CELL 13 — CROSSING-POINT ESTIMATE (FINITE-SIZE SCALING STYLE)
# Run this BELOW Cell 12
# ==============================================================================

import os
import json
import math
import numpy as np
from datetime import datetime

# We will use the isotonic-smoothed curves from Cell 12's in-memory object:
#   smoothed[d] = list of dicts with fields: p, rate_raw, rate_iso, ...
# If smoothed isn't in scope (should be), we can load it from the saved JSON.

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")

# ----------------------------------------------------------------------
# Ensure 'smoothed' exists (fallback load)
# ----------------------------------------------------------------------
if "smoothed" not in globals():
    # Try to load from the last run path printed in Cell 12 (hardcoded fallback).
    # If you ran Cell 12 under a different run_id, update the path below.
    path = "/content/shadowmap_cps_bootstrap/engine_runs/monotone_repair_run_20260215T023029/smoothed_curves.json"
    with open(path, "r") as f:
        smoothed = json.load(f)
    # keys might be strings after JSON load
    smoothed = {int(k): v for k, v in smoothed.items()}

distances = sorted(smoothed.keys())

# ----------------------------------------------------------------------
# Interpolate isotonic curves to a common fine p-grid
# ----------------------------------------------------------------------
p_min = max(min(r["p"] for r in smoothed[d]) for d in distances)
p_max = min(max(r["p"] for r in smoothed[d]) for d in distances)
p_fine = np.round(np.linspace(p_min, p_max, 121), 4)  # ~0.0012 resolution

def interp_curve(d_val, p_query):
    rows = sorted(smoothed[d_val], key=lambda r: r["p"])
    ps = [float(r["p"]) for r in rows]
    ys = [float(r["rate_iso"]) for r in rows]
    # clamp endpoints
    if p_query <= ps[0]:
        return ys[0]
    if p_query >= ps[-1]:
        return ys[-1]
    # find interval
    for i in range(len(ps)-1):
        if ps[i] <= p_query <= ps[i+1]:
            p1, p2 = ps[i], ps[i+1]
            y1, y2 = ys[i], ys[i+1]
            if p2 == p1:
                return y1
            t = (p_query - p1) / (p2 - p1)
            return y1 + t*(y2 - y1)
    return ys[-1]

# ----------------------------------------------------------------------
# Crossing estimator:
# Choose p where curves are most "tightly clustered" across d.
# We use standard deviation across distances at each p.
# ----------------------------------------------------------------------
dispersion = []
matrix = {}  # p -> {d: rate}

for p in p_fine:
    vals = []
    matrix[p] = {}
    for d in distances:
        y = interp_curve(d, float(p))
        matrix[p][d] = y
        vals.append(y)
    mean = sum(vals)/len(vals)
    sd = math.sqrt(sum((v-mean)**2 for v in vals)/len(vals))
    dispersion.append((sd, float(p), mean, vals))

dispersion.sort(key=lambda x: x[0])
best_sd, best_p, best_mean, best_vals = dispersion[0]

# ----------------------------------------------------------------------
# Report + save
# ----------------------------------------------------------------------
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"crossing_estimate_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

artifact = {
    "distances": distances,
    "p_range": [float(p_min), float(p_max)],
    "p_fine_len": len(p_fine),
    "best_p": best_p,
    "best_sd": best_sd,
    "best_mean_rate": best_mean,
    "rates_at_best_p": {d: matrix[best_p][d] for d in distances},
}

with open(os.path.join(run_dir, "crossing_estimate.json"), "w") as f:
    json.dump(artifact, f, indent=2)

# Print a small neighborhood around best_p
def neighborhood(center_p, window=0.01, step=0.002):
    ps = [round(center_p + k*step, 4) for k in range(int(-window/step), int(window/step)+1)]
    ps = [p for p in ps if p_min <= p <= p_max]
    rows = []
    for p in ps:
        vals = [interp_curve(d, p) for d in distances]
        mean = sum(vals)/len(vals)
        sd = math.sqrt(sum((v-mean)**2 for v in vals)/len(vals))
        rows.append((sd, p, mean, vals))
    rows.sort(key=lambda x: x[0])
    return rows[:10]

top10 = neighborhood(best_p, window=0.012, step=0.002)

print("="*70)
print("CELL 13 CROSSING ESTIMATE (MIN DISPERSION ACROSS DISTANCES)")
print("="*70)
print("Distances:", distances)
print(f"Common p-range: [{p_min:.3f}, {p_max:.3f}]  | fine grid size={len(p_fine)}")
print("-"*70)
print(f"Best p_cross ~= {best_p:.4f}")
print(f"Dispersion (sd across d) = {best_sd:.6f}")
print(f"Mean rate at p_cross     = {best_mean:.4f}")
print("Rates at p_cross:", {d: round(matrix[best_p][d], 4) for d in distances})
print("-"*70)
print("Top nearby candidates (lowest sd):")
for sd, p, mean, vals in top10:
    rates = {distances[i]: round(vals[i], 4) for i in range(len(distances))}
    print(f"  p={p:.4f}  sd={sd:.6f}  mean={mean:.4f}  rates={rates}")
print("-"*70)
print("Saved:", run_dir)
print("="*70)

CELL 13 CROSSING ESTIMATE (MIN DISPERSION ACROSS DISTANCES)
Distances: [5, 7, 9, 11]
Common p-range: [0.120, 0.260]  | fine grid size=121
----------------------------------------------------------------------
Best p_cross ~= 0.1643
Dispersion (sd across d) = 0.001797
Mean rate at p_cross     = 0.0511
Rates at p_cross: {5: 0.0527, 7: 0.0491, 9: 0.0495, 11: 0.0531}
----------------------------------------------------------------------
Top nearby candidates (lowest sd):
  p=0.1643  sd=0.001797  mean=0.0511  rates={5: 0.0527, 7: 0.0491, 9: 0.0495, 11: 0.0531}
  p=0.1623  sd=0.002328  mean=0.0480  rates={5: 0.0514, 7: 0.0449, 9: 0.048, 11: 0.0476}
  p=0.1663  sd=0.002751  mean=0.0542  rates={5: 0.0539, 7: 0.0534, 9: 0.051, 11: 0.0586}
  p=0.1743  sd=0.003292  mean=0.0699  rates={5: 0.0718, 7: 0.0645, 9: 0.0704, 11: 0.0731}
  p=0.1723  sd=0.003389  mean=0.0653  rates={5: 0.0646, 7: 0.063, 9: 0.0627, 11: 0.0711}
  p=0.1603  sd=0.003761  mean=0.0448  rates={5: 0.0502, 7: 0.0406, 9: 0.0465, 11:

In [ ]:

# ==============================================================================
# CELL 14 — VALIDATE CROSSING: LARGER d + TWO LOGICAL OPERATORS (LR and TB)
# Run this BELOW Cell 13
# ==============================================================================

import os, json, math, random
import numpy as np
from datetime import datetime
import networkx as nx

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"crossing_validate_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

# ----------------------------------------------------------------------
# Two logical operators on the same lattice:
# - LR: left boundary to right boundary path
# - TB: top boundary to bottom boundary path
# ----------------------------------------------------------------------
def boundaries_for(d_val):
    # nodes are relabeled 0..d^2-1 in row-major order by build_planar_lattice
    left = [n for n in range(d_val*d_val) if n % d_val == 0]
    right = [n for n in range(d_val*d_val) if n % d_val == d_val-1]
    top = [n for n in range(d_val)]                         # row 0
    bottom = [n for n in range(d_val*(d_val-1), d_val*d_val)]  # row d-1
    return left, right, top, bottom

def detect_logical_error_lr(G, total_error_edges, left_boundary, right_boundary):
    eg = nx.Graph()
    eg.add_nodes_from(G.nodes())
    eg.add_edges_from(total_error_edges)
    for l in left_boundary:
        for r in right_boundary:
            if nx.has_path(eg, l, r):
                return 1
    return 0

def detect_logical_error_tb(G, total_error_edges, top_boundary, bottom_boundary):
    eg = nx.Graph()
    eg.add_nodes_from(G.nodes())
    eg.add_edges_from(total_error_edges)
    for t in top_boundary:
        for b in bottom_boundary:
            if nx.has_path(eg, t, b):
                return 1
    return 0

def logical_failure_count_two_ops(d_val, p_error, trials=1200):
    G = build_planar_lattice(d_val)
    left, right, top, bottom = boundaries_for(d_val)

    fail_lr = 0
    fail_tb = 0

    for t in range(trials):
        seed = hash((d_val, float(p_error), t)) % (2**32)
        edge_errors = inject_edge_noise(G, float(p_error), seed)
        syndrome = compute_vertex_syndrome(G, edge_errors)
        defects = [n for n, v in syndrome.items() if v == 1]
        correction = apply_matching(G, defects)

        if correction is None:
            # treat as failure for both (uncorrectable)
            fail_lr += 1
            fail_tb += 1
            continue

        total_error_edges = [e for e, err in edge_errors.items() if err] + correction
        fail_lr += detect_logical_error_lr(G, total_error_edges, left, right)
        fail_tb += detect_logical_error_tb(G, total_error_edges, top, bottom)

    return {"k_lr": fail_lr, "k_tb": fail_tb, "n": trials,
            "rate_lr": fail_lr/trials, "rate_tb": fail_tb/trials}

# ----------------------------------------------------------------------
# Distances and p-grid centered around your estimated crossing ~0.164
# ----------------------------------------------------------------------
distances = [5, 7, 9, 11, 13, 15]
p_grid = np.round(np.linspace(0.145, 0.185, 9), 4)  # tight window
trials = 1200

print("="*70)
print("CELL 14 CROSSING VALIDATION — RUNNING")
print("="*70)
print("distances:", distances)
print("p_grid:", list(p_grid))
print("trials per point:", trials)
print("-"*70)

results = {}
for d_val in distances:
    results[d_val] = {}
    for p in p_grid:
        out = logical_failure_count_two_ops(d_val, p, trials=trials)
        results[d_val][float(p)] = out
        print(f"d={d_val:>2} p={p:.4f}  LR={out['rate_lr']:.4f}  TB={out['rate_tb']:.4f}  (n={trials})")

# ----------------------------------------------------------------------
# Crossing estimator: find p where dispersion across d is minimized, separately for LR and TB
# ----------------------------------------------------------------------
def dispersion_across_d(results, p, key):
    vals = [results[d][p][key] for d in results.keys()]
    mean = sum(vals)/len(vals)
    sd = math.sqrt(sum((v-mean)**2 for v in vals)/len(vals))
    return sd, mean, vals

p_list = [float(p) for p in p_grid]
disp_lr = []
disp_tb = []
for p in p_list:
    sd, mean, vals = dispersion_across_d(results, p, "rate_lr")
    disp_lr.append((sd, p, mean, vals))
    sd, mean, vals = dispersion_across_d(results, p, "rate_tb")
    disp_tb.append((sd, p, mean, vals))

disp_lr.sort(key=lambda x: x[0])
disp_tb.sort(key=lambda x: x[0])

best_lr = disp_lr[0]
best_tb = disp_tb[0]

artifact = {
    "distances": distances,
    "p_grid": p_list,
    "trials": trials,
    "best_lr": {"sd": best_lr[0], "p": best_lr[1], "mean": best_lr[2],
                "rates_by_d": {d: results[d][best_lr[1]]["rate_lr"] for d in distances}},
    "best_tb": {"sd": best_tb[0], "p": best_tb[1], "mean": best_tb[2],
                "rates_by_d": {d: results[d][best_tb[1]]["rate_tb"] for d in distances}},
}

with open(os.path.join(run_dir, "crossing_validate.json"), "w") as f:
    json.dump(artifact, f, indent=2)

print("="*70)
print("CELL 14 SUMMARY")
print("="*70)
print("Best LR dispersion:", {"p": best_lr[1], "sd": best_lr[0], "mean": best_lr[2],
                            "rates_by_d": artifact["best_lr"]["rates_by_d"]})
print("Best TB dispersion:", {"p": best_tb[1], "sd": best_tb[0], "mean": best_tb[2],
                            "rates_by_d": artifact["best_tb"]["rates_by_d"]})
print("Saved:", run_dir)
print("="*70)

CELL 14 CROSSING VALIDATION — RUNNING
distances: [5, 7, 9, 11, 13, 15]
p_grid: [np.float64(0.145), np.float64(0.15), np.float64(0.155), np.float64(0.16), np.float64(0.165), np.float64(0.17), np.float64(0.175), np.float64(0.18), np.float64(0.185)]
trials per point: 1200
----------------------------------------------------------------------
d= 5 p=0.1450  LR=0.0367  TB=0.0333  (n=1200)
d= 5 p=0.1500  LR=0.0392  TB=0.0392  (n=1200)
d= 5 p=0.1550  LR=0.0575  TB=0.0608  (n=1200)
d= 5 p=0.1600  LR=0.0542  TB=0.0483  (n=1200)
d= 5 p=0.1650  LR=0.0717  TB=0.0692  (n=1200)
d= 5 p=0.1700  LR=0.0758  TB=0.0775  (n=1200)
d= 5 p=0.1750  LR=0.0792  TB=0.0992  (n=1200)
d= 5 p=0.1800  LR=0.0983  TB=0.0925  (n=1200)
d= 5 p=0.1850  LR=0.0975  TB=0.1017  (n=1200)
d= 7 p=0.1450  LR=0.0292  TB=0.0275  (n=1200)
d= 7 p=0.1500  LR=0.0325  TB=0.0317  (n=1200)
d= 7 p=0.1550  LR=0.0283  TB=0.0425  (n=1200)
d= 7 p=0.1600  LR=0.0467  TB=0.0542  (n=1200)
d= 7 p=0.1650  LR=0.0525  TB=0.0575  (n=1200)
d= 7 p=0.1700  

In [ ]:

# ==============================================================================
# CELL 15 — UPGRADE MODEL: EDGE-DATA + PLAQUETTE CHECKS (Z-SYNDROME) + MWPM
# Run this BELOW Cell 14
# ==============================================================================

import networkx as nx
import random
import numpy as np
import math

# ----------------------------------------------------------------------
# Build square lattice with data on edges and plaquette checks on faces.
# We'll use a d x d grid of vertices; plaquettes are (d-1)x(d-1) faces.
# Data qubits live on edges between adjacent vertices.
# A Z-check (plaquette) is the parity of its 4 boundary edges.
# ----------------------------------------------------------------------

def build_edge_plaquette_model(d):
    # vertex grid
    V = [(i, j) for i in range(d) for j in range(d)]
    vid = {v: idx for idx, v in enumerate(V)}

    # edges (undirected) between adjacent vertices
    edges = []
    for i in range(d):
        for j in range(d):
            if i + 1 < d:
                edges.append(((i, j), (i+1, j)))
            if j + 1 < d:
                edges.append(((i, j), (i, j+1)))

    # normalize edge key
    def ekey(a, b):
        return tuple(sorted((a, b)))

    E = [ekey(a, b) for a, b in edges]

    # plaquettes (faces)
    P = []
    for i in range(d-1):
        for j in range(d-1):
            # face corners
            v00 = (i, j)
            v10 = (i+1, j)
            v01 = (i, j+1)
            v11 = (i+1, j+1)
            # boundary edges (4)
            pe = [
                ekey(v00, v10),  # left vertical
                ekey(v10, v11),  # top horizontal
                ekey(v01, v11),  # right vertical
                ekey(v00, v01),  # bottom horizontal
            ]
            P.append(((i, j), pe))

    # Build adjacency graph over plaquettes: neighbors share an edge
    PG = nx.Graph()
    p_ids = [p[0] for p in P]
    PG.add_nodes_from(p_ids)

    # map edge->plaquettes that contain it
    edge_to_faces = {}
    for pid, pe in P:
        for e in pe:
            edge_to_faces.setdefault(e, []).append(pid)

    # faces adjacent if they share any edge
    for e, faces in edge_to_faces.items():
        if len(faces) == 2:
            a, b = faces
            PG.add_edge(a, b, weight=1.0)

    return {
        "d": d,
        "V": V,
        "E": E,
        "P": P,
        "PG": PG,                # plaquette adjacency for matching
        "edge_to_faces": edge_to_faces
    }

def inject_edge_noise_edges(E, p, seed):
    rng = random.Random(seed)
    return {e: (rng.random() < p) for e in E}

def plaquette_syndrome(model, edge_errors):
    # syndrome on each plaquette = XOR of its 4 edges
    syn = {}
    for pid, pe in model["P"]:
        s = 0
        for e in pe:
            if edge_errors.get(e, False):
                s ^= 1
        syn[pid] = s
    return syn

def mwpm_on_plaquettes(PG, defects):
    if len(defects) % 2 != 0:
        return None, float("inf")
    # all-pairs shortest path lengths on PG
    spl = dict(nx.all_pairs_shortest_path_length(PG))
    K = nx.Graph()
    K.add_nodes_from(defects)
    for i in range(len(defects)):
        for j in range(i+1, len(defects)):
            a, b = defects[i], defects[j]
            dist = spl.get(a, {}).get(b, 10**6)
            K.add_edge(a, b, weight=dist)
    matching = nx.algorithms.matching.min_weight_matching(K, weight="weight")
    total = 0
    pairs = []
    for a, b in matching:
        total += K[a][b]["weight"]
        pairs.append((a, b))
    return pairs, total

# Correction reconstruction here will be "graph-theoretic" (not full path -> edges),
# but we can still define a logical error proxy:
# logical Z operator across left-right corresponds to an odd number of flipped edges
# crossing a chosen cut. We'll use the vertical cut between columns c and c+1.

def logical_error_proxy_lr_edgecut(model, edge_errors, cut_col=None):
    d = model["d"]
    if cut_col is None:
        cut_col = (d-1)//2  # middle cut

    # count flipped horizontal edges that cross between (i,cut_col) and (i,cut_col+1)
    # These are edges crossing the LR cut.
    flips = 0
    for i in range(d):
        a = (i, cut_col)
        b = (i, cut_col+1)
        e = tuple(sorted((a, b)))
        if e in edge_errors and edge_errors[e]:
            flips ^= 1  # parity
    return flips  # 1 = logical error, 0 = no logical error

def trial_lr_failure(model, p, t, trials_seed):
    seed = hash((model["d"], float(p), t, trials_seed)) % (2**32)
    edge_errors = inject_edge_noise_edges(model["E"], float(p), seed)
    syn = plaquette_syndrome(model, edge_errors)
    defects = [pid for pid, v in syn.items() if v == 1]

    # MWPM feasibility; if odd defects, treat as failure
    pairs, _ = mwpm_on_plaquettes(model["PG"], defects)
    if pairs is None:
        return 1

    # NOTE: a full correction would flip an inferred chain; we are not reconstructing it yet.
    # For a first "fidelity upgrade" step, we measure the *raw* logical error proxy parity.
    # Next step (Cell 16) will reconstruct correction chains on the dual graph.
    return logical_error_proxy_lr_edgecut(model, edge_errors)

def estimate_lr_rate(model, p, trials=2000, trials_seed=1):
    k = 0
    for t in range(trials):
        k += trial_lr_failure(model, p, t, trials_seed)
    return k, trials, k/trials

# ----------------------------------------------------------------------
# Run a compact sweep near the interesting region
# ----------------------------------------------------------------------

distances = [5, 7, 9, 11]
p_grid = np.round(np.linspace(0.145, 0.195, 11), 4)
trials = 1500
trials_seed = 7  # fixed seed to make runs comparable

print("="*70)
print("CELL 15 EDGE+PLAQUETTE MODEL — LR LOGICAL PROXY SWEEP")
print("="*70)
print("distances:", distances)
print("p_grid:", list(p_grid))
print("trials per point:", trials)
print("-"*70)

out = {}
for d in distances:
    model = build_edge_plaquette_model(d)
    out[d] = {}
    for p in p_grid:
        k, n, rate = estimate_lr_rate(model, p, trials=trials, trials_seed=trials_seed)
        out[d][float(p)] = {"k": k, "n": n, "rate": rate}
        print(f"d={d:>2} p={p:.4f}  rate={rate:.4f}  ({k}/{n})")

print("="*70)
print("RAW DICT:", out)
print("="*70)

In [ ]:

# ==============================================================================
# CELL 16 — FINITE-SIZE SCALING: p*(d) DRIFT EXTRAPOLATION
# Run BELOW previous cells
# ==============================================================================

import numpy as np
import json
import os
from datetime import datetime

# Use fixed_d_curves.json from earlier run
fixed_path = "/content/shadowmap_cps_bootstrap/engine_runs/threshold_fixed_d_run_20260215T022004/fixed_d_curves.json"

with open(fixed_path, "r") as f:
    fixed_curves = json.load(f)

# Convert keys to int
fixed_curves = {int(k): v for k, v in fixed_curves.items()}

target_rate = 0.10

def interpolate_p_star(curve, target):
    curve = sorted(curve, key=lambda r: r["p"])
    for i in range(len(curve)-1):
        r1 = curve[i]["rate"]
        r2 = curve[i+1]["rate"]
        p1 = curve[i]["p"]
        p2 = curve[i+1]["p"]
        if (r1-target)*(r2-target) < 0:
            t = (target - r1)/(r2 - r1)
            return p1 + t*(p2 - p1)
    return None

pstars = {}
for d, curve in fixed_curves.items():
    pstar = interpolate_p_star(curve, target_rate)
    pstars[d] = pstar

# Remove None values
pstars = {d:p for d,p in pstars.items() if p is not None}

ds = np.array(sorted(pstars.keys()))
ps = np.array([pstars[d] for d in ds])

# Fit p*(d) = p_c + a*(1/d)
x = 1.0/ds
A = np.vstack([np.ones_like(x), x]).T
coeffs, *_ = np.linalg.lstsq(A, ps, rcond=None)
p_c_est, a_est = coeffs

# Save artifact
BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"finite_size_fit_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

artifact = {
    "target_rate": target_rate,
    "pstars": pstars,
    "fit_model": "p*(d) = p_c + a*(1/d)",
    "p_c_estimate": float(p_c_est),
    "a_estimate": float(a_est),
}

with open(os.path.join(run_dir, "finite_size_fit.json"), "w") as f:
    json.dump(artifact, f, indent=2)

print("="*70)
print("CELL 16 FINITE-SIZE FIT COMPLETE")
print("="*70)
print("p*(d):", pstars)
print("Estimated infinite-distance threshold p_c ≈", round(p_c_est, 5))
print("Saved:", run_dir)
print("="*70)

CELL 16 FINITE-SIZE FIT COMPLETE
p*(d): {5: 0.18352941176470589, 7: 0.18657894736842107, 9: 0.19066666666666668, 11: 0.1868}
Estimated infinite-distance threshold p_c ≈ 0.19305
Saved: /content/shadowmap_cps_bootstrap/engine_runs/finite_size_fit_run_20260215T033211


In [ ]:

# ==============================================================================
# CELL 17 — ROBUSTNESS CHECK: p_c FROM MULTIPLE ISO-FAILURE TARGETS
# Uses existing smoothed_curves.json from Cell 12 (no new Monte Carlo)
# ==============================================================================

import os, json, numpy as np
from datetime import datetime

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")

# Path from your Cell 12 run (update if needed)
smoothed_path = "/content/shadowmap_cps_bootstrap/engine_runs/monotone_repair_run_20260215T023029/smoothed_curves.json"

with open(smoothed_path, "r") as f:
    smoothed = json.load(f)
smoothed = {int(k): v for k, v in smoothed.items()}

targets = [0.05, 0.10, 0.15]
distances = sorted(smoothed.keys())

def interp_p_star(rows, target):
    rows = sorted(rows, key=lambda r: r["p"])
    ps = [float(r["p"]) for r in rows]
    ys = [float(r["rate_iso"]) for r in rows]
    for i in range(len(ps)-1):
        y1, y2 = ys[i], ys[i+1]
        p1, p2 = ps[i], ps[i+1]
        if (y1-target) == 0:
            return p1
        if (y1-target)*(y2-target) < 0:
            t = (target - y1) / (y2 - y1)
            return p1 + t*(p2 - p1)
    return None

def fit_pc_from_pstars(pstars):
    ds = np.array(sorted(pstars.keys()), dtype=float)
    ps = np.array([pstars[int(d)] for d in ds], dtype=float)
    x = 1.0/ds
    A = np.vstack([np.ones_like(x), x]).T
    coeffs, *_ = np.linalg.lstsq(A, ps, rcond=None)
    p_c, a = coeffs
    return float(p_c), float(a)

summary = {}
for t in targets:
    pstars = {}
    for d in distances:
        pstar = interp_p_star(smoothed[d], t)
        if pstar is not None:
            pstars[d] = pstar
    p_c, a = fit_pc_from_pstars(pstars)
    summary[t] = {"pstars": pstars, "p_c": p_c, "a": a}

# Save artifact
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"pc_robustness_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)
out_path = os.path.join(run_dir, "pc_robustness.json")
with open(out_path, "w") as f:
    json.dump(summary, f, indent=2)

# Print concise report
print("="*70)
print("CELL 17 p_c ROBUSTNESS (MULTI-TARGET, FROM ISOTONIC CURVES)")
print("="*70)
for t in targets:
    print(f"target={t:.2f}  p_c={summary[t]['p_c']:.5f}  a={summary[t]['a']:.5f}  p*(d)={summary[t]['pstars']}")
pcs = [summary[t]["p_c"] for t in targets]
print("-"*70)
print(f"p_c spread across targets: min={min(pcs):.5f}, max={max(pcs):.5f}, range={(max(pcs)-min(pcs)):.5f}")
print("Saved:", out_path)
print("="*70)

CELL 17 p_c ROBUSTNESS (MULTI-TARGET, FROM ISOTONIC CURVES)
target=0.05  p_c=0.16800  a=-0.03509  p*(d)={5: 0.16, 7: 0.16470588235294117, 9: 0.165, 11: 0.16318181818181818}
target=0.10  p_c=0.19310  a=-0.04475  p*(d)={5: 0.18352941176470589, 7: 0.18704225352112677, 9: 0.19066666666666668, 11: 0.1868}
target=0.15  p_c=0.20153  a=0.04323  p*(d)={5: 0.21, 7: 0.2080821917808219, 9: 0.20634146341463414, 11: 0.20526315789473684}
----------------------------------------------------------------------
p_c spread across targets: min=0.16800, max=0.20153, range=0.03353
Saved: /content/shadowmap_cps_bootstrap/engine_runs/pc_robustness_run_20260215T033626/pc_robustness.json


In [ ]:

# ==============================================================================
# CELL 18 — EDGE+PLAQUETTE MODEL WITH ACTUAL CORRECTION APPLICATION (DUAL PATHS)
# Run BELOW your previous cells
# ==============================================================================

import networkx as nx
import random
import numpy as np
import os, json, math, time
from datetime import datetime

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"edge_plaquette_decode_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

# -----------------------------
# Progress helper
# -----------------------------
def pbar(i, n, prefix=""):
    width = 28
    frac = 0 if n == 0 else min(1.0, max(0.0, i/n))
    fill = int(round(frac * width))
    bar = "█"*fill + "░"*(width-fill)
    print(f"{prefix}[{bar}] {int(frac*100):>3}% ({i}/{n})", end="\r")

def pbar_done(prefix=""):
    print(" "*120, end="\r")
    print(prefix + "DONE")

# -----------------------------
# Build edge-data / plaquette-check model
# -----------------------------
def build_edge_plaquette_model(d):
    V = [(i, j) for i in range(d) for j in range(d)]

    def ekey(a, b):
        return tuple(sorted((a, b)))

    # All grid edges
    E = []
    for i in range(d):
        for j in range(d):
            if i + 1 < d:
                E.append(ekey((i, j), (i+1, j)))
            if j + 1 < d:
                E.append(ekey((i, j), (i, j+1)))

    # Plaquettes and their boundary edges
    P = []
    for i in range(d-1):
        for j in range(d-1):
            v00 = (i, j)
            v10 = (i+1, j)
            v01 = (i, j+1)
            v11 = (i+1, j+1)
            pe = [
                ekey(v00, v10),
                ekey(v10, v11),
                ekey(v01, v11),
                ekey(v00, v01),
            ]
            P.append(((i, j), pe))

    # Edge -> faces that contain it
    edge_to_faces = {}
    for pid, pe in P:
        for e in pe:
            edge_to_faces.setdefault(e, []).append(pid)

    # Dual graph over plaquettes: neighbors share an edge
    PG = nx.Graph()
    p_ids = [p[0] for p in P]
    PG.add_nodes_from(p_ids)
    for e, faces in edge_to_faces.items():
        if len(faces) == 2:
            a, b = faces
            PG.add_edge(a, b, weight=1.0)

    return {"d": d, "V": V, "E": E, "P": P, "PG": PG, "edge_to_faces": edge_to_faces}

def inject_edge_noise(E, p, seed):
    rng = random.Random(seed)
    return {e: (rng.random() < p) for e in E}

def plaquette_syndrome(model, edge_errors):
    syn = {}
    for pid, pe in model["P"]:
        s = 0
        for e in pe:
            if edge_errors.get(e, False):
                s ^= 1
        syn[pid] = s
    return syn

# -----------------------------
# MWPM on defects in dual graph
# -----------------------------
def mwpm_pairs(PG, defects):
    if len(defects) % 2 != 0:
        return None

    spl = dict(nx.all_pairs_shortest_path_length(PG))
    K = nx.Graph()
    K.add_nodes_from(defects)
    for i in range(len(defects)):
        for j in range(i+1, len(defects)):
            a, b = defects[i], defects[j]
            dist = spl.get(a, {}).get(b, 10**6)
            K.add_edge(a, b, weight=dist)

    matching = nx.algorithms.matching.min_weight_matching(K, weight="weight")
    return list(matching)

# -----------------------------
# Turn a dual-graph path into primal edges to flip as "correction"
# Each step between adjacent plaquettes crosses one shared primal edge.
# We flip that shared edge.
# -----------------------------
def shared_edge_between_faces(model, f1, f2):
    # find the unique primal edge shared by these two faces (they must be adjacent)
    # brute force via edge_to_faces map (fast enough for our sizes)
    for e, faces in model["edge_to_faces"].items():
        if len(faces) == 2:
            if (faces[0] == f1 and faces[1] == f2) or (faces[0] == f2 and faces[1] == f1):
                return e
    return None

def correction_edges_from_pair(model, f1, f2):
    PG = model["PG"]
    path = nx.shortest_path(PG, f1, f2)
    flips = []
    for i in range(len(path)-1):
        e = shared_edge_between_faces(model, path[i], path[i+1])
        if e is None:
            # should not happen if PG constructed correctly
            continue
        flips.append(e)
    return flips

def apply_correction(model, edge_errors, pairs):
    # Copy errors; toggle along correction edges
    corrected = dict(edge_errors)
    for (a, b) in pairs:
        flips = correction_edges_from_pair(model, a, b)
        for e in flips:
            corrected[e] = not corrected.get(e, False)
    return corrected

# -----------------------------
# Logical Z operator: parity of flips across a vertical cut (LR)
# Use cut between columns c and c+1
# -----------------------------
def logical_parity_lr(model, edge_errors, cut_col=None):
    d = model["d"]
    if cut_col is None:
        cut_col = (d-1)//2

    parity = 0
    for i in range(d):
        a = (i, cut_col)
        b = (i, cut_col+1)
        e = tuple(sorted((a, b)))
        if edge_errors.get(e, False):
            parity ^= 1
    return parity  # 1 => logical error

# -----------------------------
# Monte Carlo estimate with actual correction
# -----------------------------
def lr_failure_rate_after_decode(d, p, trials=1000, seed_tag=1):
    model = build_edge_plaquette_model(d)
    fail = 0
    uncorrectable = 0

    for t in range(trials):
        seed = hash((d, float(p), t, seed_tag)) % (2**32)
        edge_err = inject_edge_noise(model["E"], float(p), seed)
        syn = plaquette_syndrome(model, edge_err)
        defects = [pid for pid, v in syn.items() if v == 1]

        pairs = mwpm_pairs(model["PG"], defects)
        if pairs is None:
            uncorrectable += 1
            fail += 1
            continue

        corrected = apply_correction(model, edge_err, pairs)
        fail += logical_parity_lr(model, corrected)

    return {"rate": fail/trials, "uncorrectable": uncorrectable/trials, "k": fail, "n": trials}

# -----------------------------
# Run a compact sweep (CPU-friendly) centered near your earlier bands
# -----------------------------
distances = [5, 7, 9, 11]
p_grid = np.round(np.linspace(0.15, 0.21, 7), 4)  # coarse; we refine next
trials = 1200
seed_tag = 42

total_jobs = len(distances) * len(p_grid)
job = 0

print("="*70)
print("CELL 18 EDGE+PLAQUETTE + CORRECTION (LR) — RUNNING")
print("="*70)
print("distances:", distances)
print("p_grid:", list(p_grid))
print("trials per point:", trials)
print("-"*70)

results = {}
t0 = time.time()

for d in distances:
    results[d] = {}
    for p in p_grid:
        job += 1
        pbar(job, total_jobs, prefix="SIM ")
        out = lr_failure_rate_after_decode(d, p, trials=trials, seed_tag=seed_tag)
        results[d][float(p)] = out

pbar_done(prefix="SIM ")

elapsed = time.time() - t0

# Save
with open(os.path.join(run_dir, "edge_plaquette_decode_lr.json"), "w") as f:
    json.dump(results, f, indent=2)

print("="*70)
print("CELL 18 COMPLETE")
print("="*70)
print("Elapsed seconds:", round(elapsed, 3))
print("Saved:", run_dir)
print("Results (d -> p -> {rate, uncorrectable, k, n}):")
print(results)
print("="*70)

CELL 18 EDGE+PLAQUETTE + CORRECTION (LR) — RUNNING
distances: [5, 7, 9, 11]
p_grid: [np.float64(0.15), np.float64(0.16), np.float64(0.17), np.float64(0.18), np.float64(0.19), np.float64(0.2), np.float64(0.21)]
trials per point: 1200
----------------------------------------------------------------------
SIM DONE
CELL 18 COMPLETE
Elapsed seconds: 113.811
Saved: /content/shadowmap_cps_bootstrap/engine_runs/edge_plaquette_decode_run_20260215T034244
Results (d -> p -> {rate, uncorrectable, k, n}):
{5: {0.15: {'rate': 0.6841666666666667, 'uncorrectable': 0.5066666666666667, 'k': 821, 'n': 1200}, 0.16: {'rate': 0.6883333333333334, 'uncorrectable': 0.48, 'k': 826, 'n': 1200}, 0.17: {'rate': 0.7025, 'uncorrectable': 0.5141666666666667, 'k': 843, 'n': 1200}, 0.18: {'rate': 0.6983333333333334, 'uncorrectable': 0.4925, 'k': 838, 'n': 1200}, 0.19: {'rate': 0.7258333333333333, 'uncorrectable': 0.505, 'k': 871, 'n': 1200}, 0.2: {'rate': 0.7016666666666667, 'uncorrectable': 0.4825, 'k': 842, 'n': 1200

In [ ]:

#Your Cell 18 numbers diagnose the core issue: ~50% “uncorrectable” is exactly what you get when you require an even number of defects, but your planar code has open boundaries, so it’s normal to have an odd number of plaquette defects. In a real surface-code decoder, those “extra” defects are matched to a boundary (virtual node), not declared uncorrectable.
#So the next step is to upgrade MWPM to boundary-aware matching.
#Run this next cell under Cell 18.
# ==============================================================================
# CELL 19 — FIX DECODER: BOUNDARY-AWARE MWPM (ELIMINATE 50% "UNCORRECTABLE")
# Run BELOW Cell 18
# ==============================================================================

import networkx as nx
import random
import numpy as np
import os, json, time
from datetime import datetime

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"edge_plaquette_decode_boundary_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

# -----------------------------
# Progress helper
# -----------------------------
def pbar(i, n, prefix=""):
    width = 28
    frac = 0 if n == 0 else min(1.0, max(0.0, i/n))
    fill = int(round(frac * width))
    bar = "█"*fill + "░"*(width-fill)
    print(f"{prefix}[{bar}] {int(frac*100):>3}% ({i}/{n})", end="\r")

def pbar_done(prefix=""):
    print(" "*120, end="\r")
    print(prefix + "DONE")

# -----------------------------
# Edge-data / plaquette-check model (same as Cell 18)
# -----------------------------
def build_edge_plaquette_model(d):
    V = [(i, j) for i in range(d) for j in range(d)]

    def ekey(a, b):
        return tuple(sorted((a, b)))

    # All grid edges
    E = []
    for i in range(d):
        for j in range(d):
            if i + 1 < d:
                E.append(ekey((i, j), (i+1, j)))
            if j + 1 < d:
                E.append(ekey((i, j), (i, j+1)))

    # Plaquettes and their boundary edges
    P = []
    for i in range(d-1):
        for j in range(d-1):
            v00 = (i, j)
            v10 = (i+1, j)
            v01 = (i, j+1)
            v11 = (i+1, j+1)
            pe = [
                ekey(v00, v10),
                ekey(v10, v11),
                ekey(v01, v11),
                ekey(v00, v01),
            ]
            P.append(((i, j), pe))

    # Edge -> faces that contain it
    edge_to_faces = {}
    for pid, pe in P:
        for e in pe:
            edge_to_faces.setdefault(e, []).append(pid)

    # Dual graph over plaquettes: neighbors share an edge
    PG = nx.Graph()
    p_ids = [p[0] for p in P]
    PG.add_nodes_from(p_ids)
    for e, faces in edge_to_faces.items():
        if len(faces) == 2:
            a, b = faces
            PG.add_edge(a, b, weight=1.0)

    return {"d": d, "V": V, "E": E, "P": P, "PG": PG, "edge_to_faces": edge_to_faces}

def inject_edge_noise(E, p, seed):
    rng = random.Random(seed)
    return {e: (rng.random() < p) for e in E}

def plaquette_syndrome(model, edge_errors):
    syn = {}
    for pid, pe in model["P"]:
        s = 0
        for e in pe:
            if edge_errors.get(e, False):
                s ^= 1
        syn[pid] = s
    return syn

# -----------------------------
# Boundary-aware MWPM on plaquette defects
# Idea: allow pairing defects to a virtual boundary node with cost = dist to nearest boundary face.
# This eliminates the "odd defects => uncorrectable" failure mode.
# -----------------------------
def dist_to_boundary(face, d):
    # face is (i,j) with i,j in [0..d-2]
    i, j = face
    maxf = d - 2
    return min(i, j, maxf - i, maxf - j)

def mwpm_pairs_with_boundary(model, defects):
    PG = model["PG"]
    if len(defects) == 0:
        return []  # nothing to do

    # Precompute shortest path lengths between all plaquettes (unweighted)
    spl = dict(nx.all_pairs_shortest_path_length(PG))

    # Build complete graph over defects (+ optional boundary node B)
    K = nx.Graph()
    for f in defects:
        K.add_node(("D", f))

    # defect-defect edges
    for i in range(len(defects)):
        for j in range(i+1, len(defects)):
            a, b = defects[i], defects[j]
            dist = spl.get(a, {}).get(b, 10**6)
            K.add_edge(("D", a), ("D", b), weight=dist)

    # If odd number of defects, add a single boundary node B to absorb one defect
    # (NetworkX min_weight_matching with maxcardinality=True will then match all defect nodes.)
    if len(defects) % 2 == 1:
        K.add_node(("B", "boundary"))
        for f in defects:
            # cost to boundary ~ distance to nearest boundary face
            K.add_edge(("D", f), ("B", "boundary"), weight=dist_to_boundary(f, model["d"]))

    matching = nx.algorithms.matching.min_weight_matching(K, weight="weight", maxcardinality=True)

    # Convert matching into pairs of faces or (face, boundary)
    pairs = []
    for u, v in matching:
        if u[0] == "D" and v[0] == "D":
            pairs.append((u[1], v[1]))
        elif u[0] == "D" and v[0] == "B":
            pairs.append((u[1], None))  # defect matched to boundary
        elif u[0] == "B" and v[0] == "D":
            pairs.append((v[1], None))
        # ignore boundary-boundary (shouldn't happen with single B)

    return pairs

# -----------------------------
# Turn a dual-graph path into primal edges to flip as correction
# If paired to boundary, route to nearest boundary face via shortest path on PG to a boundary face.
# -----------------------------
def shared_edge_between_faces(model, f1, f2):
    for e, faces in model["edge_to_faces"].items():
        if len(faces) == 2:
            if (faces[0] == f1 and faces[1] == f2) or (faces[0] == f2 and faces[1] == f1):
                return e
    return None

def correction_edges_between_faces(model, f1, f2):
    PG = model["PG"]
    path = nx.shortest_path(PG, f1, f2)
    flips = []
    for i in range(len(path)-1):
        e = shared_edge_between_faces(model, path[i], path[i+1])
        if e is not None:
            flips.append(e)
    return flips

def nearest_boundary_face(face, d):
    # pick one of the closest boundary faces (ties arbitrary)
    i, j = face
    maxf = d - 2
    candidates = []
    # boundary faces have i==0 or j==0 or i==maxf or j==maxf
    for ii in [0, maxf]:
        candidates.append((ii, j))
    for jj in [0, maxf]:
        candidates.append((i, jj))
    # choose candidate with min Manhattan distance
    best = min(candidates, key=lambda f: abs(f[0]-i) + abs(f[1]-j))
    return best

def apply_correction_with_boundary(model, edge_errors, pairs):
    corrected = dict(edge_errors)
    for a, b in pairs:
        if b is None:
            # route defect to nearest boundary face
            bface = nearest_boundary_face(a, model["d"])
            flips = correction_edges_between_faces(model, a, bface)
        else:
            flips = correction_edges_between_faces(model, a, b)
        for e in flips:
            corrected[e] = not corrected.get(e, False)
    return corrected

# -----------------------------
# Logical parity (LR) after correction: vertical cut parity
# -----------------------------
def logical_parity_lr(model, edge_errors, cut_col=None):
    d = model["d"]
    if cut_col is None:
        cut_col = (d-1)//2
    parity = 0
    for i in range(d):
        a = (i, cut_col)
        b = (i, cut_col+1)
        e = tuple(sorted((a, b)))
        if edge_errors.get(e, False):
            parity ^= 1
    return parity

# -----------------------------
# Monte Carlo rate with boundary-aware decode
# -----------------------------
def lr_rate_after_boundary_decode(d, p, trials=800, seed_tag=1):
    model = build_edge_plaquette_model(d)
    fail = 0
    boundary_matched = 0

    for t in range(trials):
        seed = hash((d, float(p), t, seed_tag)) % (2**32)
        edge_err = inject_edge_noise(model["E"], float(p), seed)
        syn = plaquette_syndrome(model, edge_err)
        defects = [pid for pid, v in syn.items() if v == 1]

        pairs = mwpm_pairs_with_boundary(model, defects)
        if any(b is None for (_, b) in pairs):
            boundary_matched += 1

        corrected = apply_correction_with_boundary(model, edge_err, pairs)
        fail += logical_parity_lr(model, corrected)

    return {
        "rate": fail/trials,
        "boundary_used_frac": boundary_matched/trials,
        "k": fail,
        "n": trials
    }

# -----------------------------
# Run sweep (same p-grid as Cell 18 for comparability)
# -----------------------------
distances = [5, 7, 9, 11]
p_grid = np.round(np.linspace(0.15, 0.21, 7), 4)
trials = 1200
seed_tag = 42

total_jobs = len(distances) * len(p_grid)
job = 0

print("="*70)
print("CELL 19 BOUNDARY-AWARE DECODE (LR) — RUNNING")
print("="*70)
print("distances:", distances)
print("p_grid:", list(p_grid))
print("trials per point:", trials)
print("-"*70)

results = {}
t0 = time.time()

for d in distances:
    results[d] = {}
    for p in p_grid:
        job += 1
        pbar(job, total_jobs, prefix="SIM ")
        out = lr_rate_after_boundary_decode(d, p, trials=trials, seed_tag=seed_tag)
        results[d][float(p)] = out

pbar_done(prefix="SIM ")
elapsed = time.time() - t0

with open(os.path.join(run_dir, "edge_plaquette_decode_lr_boundary.json"), "w") as f:
    json.dump(results, f, indent=2)

print("="*70)
print("CELL 19 COMPLETE")
print("="*70)
print("Elapsed seconds:", round(elapsed, 3))
print("Saved:", run_dir)
print("Results (d -> p -> {rate, boundary_used_frac, k, n}):")
print(results)
print("="*70)

CELL 19 BOUNDARY-AWARE DECODE (LR) — RUNNING
distances: [5, 7, 9, 11]
p_grid: [np.float64(0.15), np.float64(0.16), np.float64(0.17), np.float64(0.18), np.float64(0.19), np.float64(0.2), np.float64(0.21)]
trials per point: 1200
----------------------------------------------------------------------


TypeError: min_weight_matching() got an unexpected keyword argument 'maxcardinality'

In [ ]:

# ==============================================================================
# CELL 19B — PATCH: NetworkX min_weight_matching() (no maxcardinality kwarg)
# Then re-run the boundary-aware sweep
# Run BELOW the errored Cell 19
# ==============================================================================

import networkx as nx
import numpy as np
import os, json, time
from datetime import datetime

# -----------------------------
# Progress helper (reuse)
# -----------------------------
def pbar(i, n, prefix=""):
    width = 28
    frac = 0 if n == 0 else min(1.0, max(0.0, i/n))
    fill = int(round(frac * width))
    bar = "█"*fill + "░"*(width-fill)
    print(f"{prefix}[{bar}] {int(frac*100):>3}% ({i}/{n})", end="\r")

def pbar_done(prefix=""):
    print(" "*120, end="\r")
    print(prefix + "DONE")

# -----------------------------
# Patch ONLY this function: remove unsupported kwarg
# -----------------------------
def mwpm_pairs_with_boundary(model, defects):
    PG = model["PG"]
    if len(defects) == 0:
        return []

    # Precompute shortest path lengths between all plaquettes (unweighted)
    spl = dict(nx.all_pairs_shortest_path_length(PG))

    K = nx.Graph()
    for f in defects:
        K.add_node(("D", f))

    # defect-defect edges
    for i in range(len(defects)):
        for j in range(i+1, len(defects)):
            a, b = defects[i], defects[j]
            dist = spl.get(a, {}).get(b, 10**6)
            K.add_edge(("D", a), ("D", b), weight=dist)

    # If odd, add one boundary node to make node count even
    if len(defects) % 2 == 1:
        K.add_node(("B", "boundary"))
        for f in defects:
            K.add_edge(("D", f), ("B", "boundary"), weight=dist_to_boundary(f, model["d"]))

    # NetworkX in your env: no maxcardinality kwarg; default returns a max-cardinality MW matching
    matching = nx.algorithms.matching.min_weight_matching(K, weight="weight")

    pairs = []
    for u, v in matching:
        if u[0] == "D" and v[0] == "D":
            pairs.append((u[1], v[1]))
        elif u[0] == "D" and v[0] == "B":
            pairs.append((u[1], None))
        elif u[0] == "B" and v[0] == "D":
            pairs.append((v[1], None))
    return pairs

print("Patched mwpm_pairs_with_boundary(): removed unsupported maxcardinality kwarg.")

# -----------------------------
# Re-run the sweep (same settings as Cell 19)
# -----------------------------
BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"edge_plaquette_decode_boundary_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

distances = [5, 7, 9, 11]
p_grid = np.round(np.linspace(0.15, 0.21, 7), 4)
trials = 1200
seed_tag = 42

total_jobs = len(distances) * len(p_grid)
job = 0

print("="*70)
print("CELL 19B BOUNDARY-AWARE DECODE (LR) — RUNNING")
print("="*70)
print("run_id:", run_id)
print("distances:", distances)
print("p_grid:", list(p_grid))
print("trials per point:", trials)
print("-"*70)

results = {}
t0 = time.time()

for d in distances:
    results[d] = {}
    for p in p_grid:
        job += 1
        pbar(job, total_jobs, prefix="SIM ")
        out = lr_rate_after_boundary_decode(d, p, trials=trials, seed_tag=seed_tag)
        results[d][float(p)] = out

pbar_done(prefix="SIM ")
elapsed = time.time() - t0

with open(os.path.join(run_dir, "edge_plaquette_decode_lr_boundary.json"), "w") as f:
    json.dump(results, f, indent=2)

print("="*70)
print("CELL 19B COMPLETE")
print("="*70)
print("Elapsed seconds:", round(elapsed, 3))
print("Saved:", run_dir)
print("Results (d -> p -> {rate, boundary_used_frac, k, n}):")
print(results)
print("="*70)

Patched mwpm_pairs_with_boundary(): removed unsupported maxcardinality kwarg.
CELL 19B BOUNDARY-AWARE DECODE (LR) — RUNNING
run_id: 20260215T040106
distances: [5, 7, 9, 11]
p_grid: [np.float64(0.15), np.float64(0.16), np.float64(0.17), np.float64(0.18), np.float64(0.19), np.float64(0.2), np.float64(0.21)]
trials per point: 1200
----------------------------------------------------------------------
SIM DONE
CELL 19B COMPLETE
Elapsed seconds: 268.795
Saved: /content/shadowmap_cps_bootstrap/engine_runs/edge_plaquette_decode_boundary_run_20260215T040106
Results (d -> p -> {rate, boundary_used_frac, k, n}):
{5: {0.15: {'rate': 0.36583333333333334, 'boundary_used_frac': 0.5066666666666667, 'k': 439, 'n': 1200}, 0.16: {'rate': 0.39666666666666667, 'boundary_used_frac': 0.48, 'k': 476, 'n': 1200}, 0.17: {'rate': 0.3933333333333333, 'boundary_used_frac': 0.5141666666666667, 'k': 472, 'n': 1200}, 0.18: {'rate': 0.4091666666666667, 'boundary_used_frac': 0.4925, 'k': 491, 'n': 1200}, 0.19: {'rate'

In [ ]:

# ==============================================================================
# CELL 20 — DIAGNOSTIC: FAILURE RATE CONDITIONED ON BOUNDARY USE
# Run BELOW Cell 19B
# ==============================================================================

import numpy as np
import os, json, time
from datetime import datetime

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"boundary_conditioned_diag_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

# We reuse functions from Cell 19/19B in-memory:
# - build_edge_plaquette_model
# - inject_edge_noise
# - plaquette_syndrome
# - mwpm_pairs_with_boundary (patched)
# - apply_correction_with_boundary
# - logical_parity_lr

def lr_rate_conditioned(d, p, trials=1200, seed_tag=99):
    model = build_edge_plaquette_model(d)
    fail_b0 = 0
    n_b0 = 0
    fail_b1 = 0
    n_b1 = 0

    for t in range(trials):
        seed = hash((d, float(p), t, seed_tag)) % (2**32)
        edge_err = inject_edge_noise(model["E"], float(p), seed)
        syn = plaquette_syndrome(model, edge_err)
        defects = [pid for pid, v in syn.items() if v == 1]

        pairs = mwpm_pairs_with_boundary(model, defects)
        used_boundary = any(b is None for (_, b) in pairs)

        corrected = apply_correction_with_boundary(model, edge_err, pairs)
        is_fail = logical_parity_lr(model, corrected)

        if used_boundary:
            n_b1 += 1
            fail_b1 += is_fail
        else:
            n_b0 += 1
            fail_b0 += is_fail

    out = {
        "p": float(p),
        "d": int(d),
        "trials": int(trials),
        "boundary_used_frac": (n_b1 / trials),
        "rate_all": ((fail_b0 + fail_b1) / trials),
        "rate_no_boundary": (fail_b0 / n_b0) if n_b0 else None,
        "n_no_boundary": int(n_b0),
        "rate_with_boundary": (fail_b1 / n_b1) if n_b1 else None,
        "n_with_boundary": int(n_b1),
    }
    return out

distances = [5, 7, 9, 11]
p_grid = [0.15, 0.17, 0.19, 0.21]
trials = 2000

print("="*70)
print("CELL 20 CONDITIONED DIAGNOSTIC — RUNNING")
print("="*70)
print("distances:", distances)
print("p_grid:", p_grid)
print("trials:", trials)
print("-"*70)

results = {}
t0 = time.time()
for d in distances:
    results[d] = {}
    for p in p_grid:
        out = lr_rate_conditioned(d, p, trials=trials, seed_tag=123)
        results[d][p] = out
        print(
            f"d={d:>2} p={p:.2f}  all={out['rate_all']:.3f}  "
            f"b0={out['rate_no_boundary']:.3f} (n={out['n_no_boundary']})  "
            f"b1={out['rate_with_boundary']:.3f} (n={out['n_with_boundary']})  "
            f"bfrac={out['boundary_used_frac']:.3f}"
        )

elapsed = time.time() - t0

with open(os.path.join(run_dir, "conditioned_diag.json"), "w") as f:
    json.dump(results, f, indent=2)

print("-"*70)
print("Elapsed seconds:", round(elapsed, 3))
print("Saved:", run_dir)
print("="*70)

CELL 20 CONDITIONED DIAGNOSTIC — RUNNING
distances: [5, 7, 9, 11]
p_grid: [0.15, 0.17, 0.19, 0.21]
trials: 2000
----------------------------------------------------------------------
d= 5 p=0.15  all=0.382  b0=0.378 (n=1014)  b1=0.385 (n=986)  bfrac=0.493
d= 5 p=0.17  all=0.402  b0=0.405 (n=1024)  b1=0.398 (n=976)  bfrac=0.488
d= 5 p=0.19  all=0.445  b0=0.456 (n=1014)  b1=0.434 (n=986)  bfrac=0.493
d= 5 p=0.21  all=0.455  b0=0.457 (n=976)  b1=0.453 (n=1024)  bfrac=0.512
d= 7 p=0.15  all=0.399  b0=0.397 (n=1021)  b1=0.400 (n=979)  bfrac=0.489
d= 7 p=0.17  all=0.466  b0=0.465 (n=986)  b1=0.467 (n=1014)  bfrac=0.507
d= 7 p=0.19  all=0.475  b0=0.484 (n=974)  b1=0.467 (n=1026)  bfrac=0.513
d= 7 p=0.21  all=0.472  b0=0.464 (n=981)  b1=0.481 (n=1019)  bfrac=0.509
d= 9 p=0.15  all=0.441  b0=0.436 (n=995)  b1=0.445 (n=1005)  bfrac=0.502
d= 9 p=0.17  all=0.475  b0=0.491 (n=995)  b1=0.460 (n=1005)  bfrac=0.502
d= 9 p=0.19  all=0.495  b0=0.489 (n=999)  b1=0.500 (n=1001)  bfrac=0.500
d= 9 p=0.21  a

In [ ]:

# ==============================================================================
# CELL 21 — FIX LOGICAL ERROR METRIC: USE RESIDUAL-CHAIN CONNECTIVITY (HOMOLOGY PROXY)
# Run BELOW Cell 20
# ==============================================================================

import networkx as nx
import random
import numpy as np
import os, json, time
from datetime import datetime

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"logical_metric_fix_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

# Reuse functions from Cell 19B in memory:
# - build_edge_plaquette_model
# - inject_edge_noise
# - plaquette_syndrome
# - mwpm_pairs_with_boundary (patched)
# - apply_correction_with_boundary

def vertex_boundaries_for_primal(d):
    # vertices are (i,j), i=row, j=col, with i,j in 0..d-1
    top = [(0, j) for j in range(d)]
    bottom = [(d-1, j) for j in range(d)]
    left = [(i, 0) for i in range(d)]
    right = [(i, d-1) for i in range(d)]
    return left, right, top, bottom

def residual_error_edges(edge_err, corrected_err):
    # residual = error XOR correction (edges where boolean differs)
    res = []
    for e in edge_err.keys():
        if edge_err.get(e, False) ^ corrected_err.get(e, False):
            res.append(e)
    return res

def has_spanning_path_primal(model, residual_edges, side="LR"):
    # Build primal graph of residual edges on vertex nodes
    d = model["d"]
    left, right, top, bottom = vertex_boundaries_for_primal(d)

    Gres = nx.Graph()
    Gres.add_nodes_from(model["V"])
    # edges are between vertex tuples
    Gres.add_edges_from(residual_edges)

    if side == "LR":
        # LR logical operator: connect LEFT boundary to RIGHT boundary
        sources = left
        targets = right
    else:
        # TB logical operator: connect TOP boundary to BOTTOM boundary
        sources = top
        targets = bottom

    for s in sources:
        # quick prune: if s isolated in residual graph, skip
        if Gres.degree(s) == 0:
            continue
        for t in targets:
            if Gres.degree(t) == 0:
                continue
            if nx.has_path(Gres, s, t):
                return 1
    return 0

def decode_and_score(model, p, t, seed_tag=7, side="LR"):
    seed = hash((model["d"], float(p), t, seed_tag, side)) % (2**32)
    edge_err = inject_edge_noise(model["E"], float(p), seed)
    syn = plaquette_syndrome(model, edge_err)
    defects = [pid for pid, v in syn.items() if v == 1]

    pairs = mwpm_pairs_with_boundary(model, defects)
    corrected = apply_correction_with_boundary(model, edge_err, pairs)

    # Residual chain after correction
    res_edges = residual_error_edges(edge_err, corrected)

    # Logical error = residual chain spans chosen boundaries
    return has_spanning_path_primal(model, res_edges, side=side), any(b is None for (_, b) in pairs)

def sweep_rates(distances, p_grid, trials=1200, side="LR"):
    out = {}
    total = len(distances) * len(p_grid)
    done = 0

    def pbar(i, n, prefix=""):
        width = 28
        frac = 0 if n == 0 else min(1.0, max(0.0, i/n))
        fill = int(round(frac * width))
        bar = "█"*fill + "░"*(width-fill)
        print(f"{prefix}[{bar}] {int(frac*100):>3}% ({i}/{n})", end="\r")

    def pbar_done(prefix=""):
        print(" "*120, end="\r")
        print(prefix + "DONE")

    t0 = time.time()
    for d in distances:
        model = build_edge_plaquette_model(d)
        out[d] = {}
        for p in p_grid:
            fail = 0
            b_used = 0
            for t in range(trials):
                f, used_b = decode_and_score(model, p, t, seed_tag=42, side=side)
                fail += f
                b_used += 1 if used_b else 0
            done += 1
            pbar(done, total, prefix=f"{side} ")
            out[d][float(p)] = {
                "rate": fail / trials,
                "boundary_used_frac": b_used / trials,
                "k": fail,
                "n": trials
            }
    pbar_done(prefix=f"{side} ")
    elapsed = time.time() - t0
    return out, elapsed

# Settings
distances = [5, 7, 9, 11]
p_grid = np.round(np.linspace(0.05, 0.20, 7), 4)  # include lower p to see distance help
trials = 1200

print("="*70)
print("CELL 21 LOGICAL METRIC FIX — RUNNING")
print("="*70)
print("distances:", distances)
print("p_grid:", list(p_grid))
print("trials per point:", trials)
print("-"*70)

# Run LR and TB to check anisotropy immediately
lr_res, lr_time = sweep_rates(distances, p_grid, trials=trials, side="LR")
tb_res, tb_time = sweep_rates(distances, p_grid, trials=trials, side="TB")

artifact = {
    "distances": distances,
    "p_grid": [float(p) for p in p_grid],
    "trials": trials,
    "LR": lr_res,
    "TB": tb_res,
    "elapsed_seconds": {"LR": lr_time, "TB": tb_time}
}

with open(os.path.join(run_dir, "logical_metric_fixed_results.json"), "w") as f:
    json.dump(artifact, f, indent=2)

print("="*70)
print("CELL 21 COMPLETE")
print("="*70)
print("Saved:", run_dir)
print("LR rates:", lr_res)
print("TB rates:", tb_res)
print("="*70)

CELL 21 LOGICAL METRIC FIX — RUNNING
distances: [5, 7, 9, 11]
p_grid: [np.float64(0.05), np.float64(0.075), np.float64(0.1), np.float64(0.125), np.float64(0.15), np.float64(0.175), np.float64(0.2)]
trials per point: 1200
----------------------------------------------------------------------
LR DONE
TB DONE
CELL 21 COMPLETE
Saved: /content/shadowmap_cps_bootstrap/engine_runs/logical_metric_fix_run_20260215T043616
LR rates: {5: {0.05: {'rate': 0.0, 'boundary_used_frac': 0.42083333333333334, 'k': 0, 'n': 1200}, 0.075: {'rate': 0.0, 'boundary_used_frac': 0.4775, 'k': 0, 'n': 1200}, 0.1: {'rate': 0.0, 'boundary_used_frac': 0.4658333333333333, 'k': 0, 'n': 1200}, 0.125: {'rate': 0.0016666666666666668, 'boundary_used_frac': 0.495, 'k': 2, 'n': 1200}, 0.15: {'rate': 0.005, 'boundary_used_frac': 0.5308333333333334, 'k': 6, 'n': 1200}, 0.175: {'rate': 0.0016666666666666668, 'boundary_used_frac': 0.5, 'k': 2, 'n': 1200}, 0.2: {'rate': 0.0025, 'boundary_used_frac': 0.5091666666666667, 'k': 3, 'n':

In [41]:
# ==============================================================================
# CELL 22 — THE TOUCHDOWN: TRUE TOPOLOGICAL PLANAR CODE (EXACT MOD-2 HOMOLOGY)
# Run BELOW Cell 21
# ==============================================================================

import networkx as nx
import numpy as np
import os, json, time
from datetime import datetime

# ------------------------------------------------------------------------------
# EXACT O(1) ANALYTICAL LATTICE GEOMETRY
# ------------------------------------------------------------------------------

def ekey(u, v):
    if u == 'B': return (v, 'B')
    if v == 'B': return (u, 'B')
    return tuple(sorted((u, v)))

def get_dist(u, v, d):
    if u == 'B' and v == 'B': return 0
    if u == 'B': u, v = v, u
    if v == 'B':
        return min(u[1] + 1, d - 1 - u[1])
    return abs(u[0] - v[0]) + abs(u[1] - v[1])

def get_correction_edges(u, v, d):
    edges = []
    if u == 'B' and v == 'B': return []
    if u == 'B': u, v = v, u

    r, c = u
    if v == 'B':
        if c + 1 <= d - 1 - c:  # Route Left
            for col in range(c - 1, -1, -1):
                edges.append(('H', r, col))
            edges.append(('L', r))
        else:                   # Route Right
            for col in range(c, d - 2):
                edges.append(('H', r, col))
            edges.append(('R', r))
        return edges

    r2, c2 = v
    step_r = 1 if r2 > r else -1
    for curr_r in range(r, r2, step_r):
        r_edge = curr_r if step_r == 1 else curr_r - 1
        edges.append(('V', r_edge, c))

    step_c = 1 if c2 > c else -1
    for curr_c in range(c, c2, step_c):
        c_edge = curr_c if step_c == 1 else curr_c - 1
        edges.append(('H', r2, c_edge))

    return edges

# ------------------------------------------------------------------------------
# RIGOROUS PLANAR CODE DECODER (MOD-2 HOMOLOGY)
# ------------------------------------------------------------------------------

def run_true_surface_code(d, p, trials=1500, seed_tag=42):
    rng = np.random.RandomState(hash((d, float(p), seed_tag)) % (2**32))

    # All Data Qubits (Edges of the exact matching graph)
    E = []
    for r in range(d - 1):
        for c in range(d - 1):
            E.append(('V', r, c))
    for r in range(d):
        for c in range(d - 2):
            E.append(('H', r, c))
    for r in range(d):
        E.append(('L', r))
        E.append(('R', r))

    # Logical Z Cut (A vertical slice down the left edge)
    left_cut = {('L', r) for r in range(d)}

    fail = 0
    for t in range(trials):
        # 1. Inject Noise
        err_mask = rng.rand(len(E)) < p
        error_edges = {E[i] for i in range(len(E)) if err_mask[i]}

        # 2. Extract Syndrome (Parity of incident errors)
        syndrome = {}
        for edge in error_edges:
            if edge[0] == 'V':
                _, r, c = edge
                syndrome[(r, c)] = syndrome.get((r, c), 0) ^ 1
                syndrome[(r+1, c)] = syndrome.get((r+1, c), 0) ^ 1
            elif edge[0] == 'H':
                _, r, c = edge
                syndrome[(r, c)] = syndrome.get((r, c), 0) ^ 1
                syndrome[(r, c+1)] = syndrome.get((r, c+1), 0) ^ 1
            elif edge[0] == 'L':
                _, r = edge
                syndrome[(r, 0)] = syndrome.get((r, 0), 0) ^ 1
            elif edge[0] == 'R':
                _, r = edge
                syndrome[(r, d-2)] = syndrome.get((r, d-2), 0) ^ 1

        defects = [node for node, s in syndrome.items() if s == 1]

        # 3. MWPM with Perfect Virtual Shadows
        correction_edges = set()
        if defects:
            K = nx.Graph()
            for i, u in enumerate(defects):
                shadow = f"s_{i}"
                K.add_node(i)
                K.add_node(shadow)
                K.add_edge(i, shadow, weight=get_dist(u, 'B', d))

                for j in range(i+1, len(defects)):
                    v = defects[j]
                    K.add_edge(i, j, weight=get_dist(u, v, d))
                    K.add_edge(shadow, f"s_{j}", weight=0) # Shadows pair for free

            matching = nx.algorithms.matching.min_weight_matching(K, weight="weight")

            # 4. Mod-2 Reconstruct
            for a, b in matching:
                if str(a).startswith("s_") and str(b).startswith("s_"): continue

                node_a = 'B' if str(a).startswith("s_") else defects[a]
                node_b = 'B' if str(b).startswith("s_") else defects[b]

                for e in get_correction_edges(node_a, node_b, d):
                    if e in correction_edges:
                        correction_edges.remove(e) # Mod-2 Cancellation!
                    else:
                        correction_edges.add(e)

        # 5. Mod-2 Homology Evaluation (Logical Parity)
        residual = error_edges.symmetric_difference(correction_edges)
        crossings = sum(1 for e in residual if e in left_cut)

        if crossings % 2 == 1:
            fail += 1

    return fail / trials

# ------------------------------------------------------------------------------
# THE PHASE TRANSITION SWEEP
# ------------------------------------------------------------------------------

def pbar(i, n, prefix=""):
    frac = 0 if n == 0 else min(1.0, max(0.0, i/n))
    fill = int(round(frac * 28))
    bar = "█"*fill + " "*(28-fill)
    print(f"{prefix} [{bar}] {int(frac*100):>3}% ({i}/{n})", end="\r")

BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"true_topological_run_{run_id}")
os.makedirs(run_dir, exist_ok=True)

distances = [5, 7, 9, 11]
# Standard Unrotated Surface Code threshold is ~10.3%.
# We sweep perfectly over the threshold to see the absolute inversion.
p_grid = [0.08, 0.09, 0.10, 0.11, 0.12]
trials = 2500

print("="*70)
print("CELL 22 - THE TOPOLOGICAL TOUCHDOWN RUNNING")
print("="*70)
print("distances:", distances)
print("p_grid:", list(p_grid))
print("trials per point:", trials)
print("EXPECTED THRESHOLD: ~0.103")
print("-"*70)

results = {}
total_jobs = len(distances) * len(p_grid)
job = 0
t0 = time.time()

for d in distances:
    results[d] = {}
    for p in p_grid:
        job += 1
        pbar(job, total_jobs, prefix="SIM ")
        rate = run_true_surface_code(d, p, trials=trials, seed_tag=42)
        results[d][float(p)] = rate

print("\n" + "-"*70)
elapsed = time.time() - t0

header = "d \\ p | " + " | ".join([f"{p:>6.3f}" for p in p_grid])
print(header)
print("-" * len(header))
for d in distances:
    parts = [f"{results[d][float(p)]:.4f}" for p in p_grid]
    print(f"{d:>5} | " + " | ".join(parts))
print("="*70)
print(f"Elapsed seconds: {elapsed:.3f}")
print("Saved to:", run_dir)
print("="*70)

with open(os.path.join(run_dir, "true_topological_results.json"), "w") as f:
    json.dump(results, f, indent=2)

CELL 22 - THE TOPOLOGICAL TOUCHDOWN RUNNING
distances: [5, 7, 9, 11]
p_grid: [0.08, 0.09, 0.1, 0.11, 0.12]
trials per point: 2500
EXPECTED THRESHOLD: ~0.103
----------------------------------------------------------------------

----------------------------------------------------------------------
d \ p |  0.080 |  0.090 |  0.100 |  0.110 |  0.120
--------------------------------------------------
    5 | 0.0884 | 0.1156 | 0.1468 | 0.1812 | 0.2012
    7 | 0.0764 | 0.0984 | 0.1424 | 0.1848 | 0.2192
    9 | 0.0528 | 0.0960 | 0.1448 | 0.1856 | 0.2212
   11 | 0.0520 | 0.0928 | 0.1292 | 0.1788 | 0.2560
Elapsed seconds: 226.989
Saved to: /content/shadowmap_cps_bootstrap/engine_runs/true_topological_run_20260215T045456


In [42]:
# ==============================================================================
# CELL 23 — THE GRAND FINALE: TRUE CPS POLICY OPTIMIZATION ON FROZEN NOISE
# Run BELOW Cell 22
# ==============================================================================

import networkx as nx
import numpy as np
import random, os, json, time
from datetime import datetime

print("="*70)
print("CELL 23: HARDWARE-SOFTWARE CO-DESIGN (CPS OPTIMIZER)")
print("="*70)

# ------------------------------------------------------------------------------
# 1. THE FROZEN WORLD (FIXED ENVIRONMENT WITH ANISOTROPIC DEFECTS)
# ------------------------------------------------------------------------------
d = 7
p_V = 0.12  # 12% error on vertical links (DEFECTIVE)
p_H = 0.04  # 4% error on horizontal/boundary links (GOOD)
trials = 1000
seed_crn = 42

print(f"Generating Common Random Numbers (CRN) for {trials} trials...")
print(f"Hardware noise frozen at p_V={p_V}, p_H={p_H}")

rng = np.random.RandomState(seed_crn)
E = []
p_mask_probs = []

for r in range(d - 1):
    for c in range(d - 1):
        E.append(('V', r, c)); p_mask_probs.append(p_V)
for r in range(d):
    for c in range(d - 2):
        E.append(('H', r, c)); p_mask_probs.append(p_H)
for r in range(d):
    E.append(('L', r)); p_mask_probs.append(p_H)
    E.append(('R', r)); p_mask_probs.append(p_H)

p_mask_probs = np.array(p_mask_probs)
left_cut = {('L', r) for r in range(d)}

crn_dataset = []
for t in range(trials):
    err_mask = rng.rand(len(E)) < p_mask_probs
    error_edges = {E[i] for i in range(len(E)) if err_mask[i]}

    syndrome = {}
    for edge in error_edges:
        if edge[0] == 'V':
            _, r, c = edge
            syndrome[(r, c)] = syndrome.get((r, c), 0) ^ 1
            syndrome[(r+1, c)] = syndrome.get((r+1, c), 0) ^ 1
        elif edge[0] == 'H':
            _, r, c = edge
            syndrome[(r, c)] = syndrome.get((r, c), 0) ^ 1
            syndrome[(r, c+1)] = syndrome.get((r, c+1), 0) ^ 1
        elif edge[0] == 'L':
            _, r = edge
            syndrome[(r, 0)] = syndrome.get((r, 0), 0) ^ 1
        elif edge[0] == 'R':
            _, r = edge
            syndrome[(r, d-2)] = syndrome.get((r, d-2), 0) ^ 1

    defects = [node for node, s in syndrome.items() if s == 1]
    crn_dataset.append((error_edges, defects))

print("CRN Generation Complete. Environment Frozen.\n")

# ------------------------------------------------------------------------------
# 2. THE POLICY (PARAMETRIZED DECODER EVALUATION)
# ------------------------------------------------------------------------------
def get_weighted_dist(u, v, d_val, w_V, w_H):
    if u == 'B' and v == 'B': return 0
    if u == 'B': u, v = v, u
    if v == 'B':
        return min(u[1] + 1, d_val - 1 - u[1]) * w_H
    return abs(u[0] - v[0]) * w_V + abs(u[1] - v[1]) * w_H

def get_correction_edges(u, v, d_val):
    edges = []
    if u == 'B' and v == 'B': return []
    if u == 'B': u, v = v, u
    r, c = u
    if v == 'B':
        if c + 1 <= d_val - 1 - c:
            for col in range(c - 1, -1, -1): edges.append(('H', r, col))
            edges.append(('L', r))
        else:
            for col in range(c, d_val - 2): edges.append(('H', r, col))
            edges.append(('R', r))
        return edges
    r2, c2 = v
    step_r = 1 if r2 > r else -1
    for curr_r in range(r, r2, step_r):
        edges.append(('V', curr_r if step_r == 1 else curr_r - 1, c))
    step_c = 1 if c2 > c else -1
    for curr_c in range(c, c2, step_c):
        edges.append(('H', r2, curr_c if step_c == 1 else curr_c - 1))
    return edges

def evaluate_policy(theta):
    w_V = max(0.01, theta['w_V'])
    w_H = max(0.01, theta['w_H'])

    fail = 0
    for error_edges, defects in crn_dataset:
        correction_edges = set()
        if defects:
            K = nx.Graph()
            for i, u in enumerate(defects):
                shadow = f"s_{i}"
                K.add_node(i); K.add_node(shadow)
                K.add_edge(i, shadow, weight=get_weighted_dist(u, 'B', d, w_V, w_H))
                for j in range(i+1, len(defects)):
                    v = defects[j]
                    K.add_edge(i, j, weight=get_weighted_dist(u, v, d, w_V, w_H))
                    K.add_edge(shadow, f"s_{j}", weight=0)

            matching = nx.algorithms.matching.min_weight_matching(K, weight="weight")

            for a, b in matching:
                if str(a).startswith("s_") and str(b).startswith("s_"): continue
                node_a = 'B' if str(a).startswith("s_") else defects[a]
                node_b = 'B' if str(b).startswith("s_") else defects[b]
                for e in get_correction_edges(node_a, node_b, d):
                    if e in correction_edges: correction_edges.remove(e)
                    else: correction_edges.add(e)

        residual = error_edges.symmetric_difference(correction_edges)
        if sum(1 for e in residual if e in left_cut) % 2 == 1:
            fail += 1

    return fail / trials

# ------------------------------------------------------------------------------
# 3. THE CPS OPTIMIZER LOOP
# ------------------------------------------------------------------------------

# BASELINE: The decoder doesn't know the hardware is defective (isotropic weights)
print("Evaluating Naive Baseline Decoder (w_V=1.0, w_H=1.0)...")
baseline_rate = evaluate_policy({'w_V': 1.0, 'w_H': 1.0})
print(f" -> Baseline Logical Failure Rate: {baseline_rate:.4f}\n")

pop_size = 12
generations = 8
retain = 4

# Start with a population clustered around 1.0 to see it evolve
population = [{'w_V': random.uniform(0.5, 1.5), 'w_H': random.uniform(0.5, 1.5)} for _ in range(pop_size)]

print("Starting CPS Evolutionary Engine...")
t0 = time.time()
best_history = []

for gen in range(generations):
    scored = []
    for theta in population:
        rate = evaluate_policy(theta)
        scored.append((rate, theta))

    scored.sort(key=lambda x: x[0])
    survivors = scored[:retain]

    best_rate, best_theta = survivors[0]
    best_history.append((best_rate, best_theta))

    print(f"GEN {gen+1:02d}/{generations} | Best Fail Rate: {best_rate:.4f} | Weights: w_V={best_theta['w_V']:.3f}, w_H={best_theta['w_H']:.3f}")

    new_pop = [s[1] for s in survivors]
    while len(new_pop) < pop_size:
        parent = random.choice(survivors)[1]
        child = {
            'w_V': max(0.01, parent['w_V'] + random.uniform(-0.2, 0.2)),
            'w_H': max(0.01, parent['w_H'] + random.uniform(-0.2, 0.2))
        }
        new_pop.append(child)
    population = new_pop

elapsed = time.time() - t0
opt_theta = best_history[-1][1]

theory_V = np.log((1 - p_V) / p_V)
theory_H = np.log((1 - p_H) / p_H)
theory_ratio = theory_V / theory_H
ai_ratio = opt_theta['w_V'] / opt_theta['w_H']

print("\n" + "="*70)
print(f"CPS OPTIMIZATION COMPLETE IN {elapsed:.1f}s")
print("="*70)
print(f"NAIVE DECODER FAILURE RATE:  {baseline_rate:.4f} (Uniform Weights)")
print(f"CPS OPTIMIZED FAILURE RATE:  {best_history[-1][0]:.4f} (Learned Hardware Asymmetry)")
if baseline_rate > 0:
    print(f"ERROR REDUCTION ACHIEVED:    {((baseline_rate - best_history[-1][0])/baseline_rate)*100:.1f}%")
print("-" * 70)
print(f"AI Discovered w_V / w_H ratio:           {ai_ratio:.3f}")
print(f"Theoretical Entropy Ratio (Shannon limit): {theory_ratio:.3f}")
print("="*70)

# Save artifact
BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
run_id = datetime.now().strftime("%Y%m%dT%H%M%S")
run_dir = os.path.join(ENGINE_DIR, f"cps_policy_optimization_{run_id}")
os.makedirs(run_dir, exist_ok=True)

with open(os.path.join(run_dir, "policy_opt.json"), "w") as f:
    json.dump({
        "baseline_rate": baseline_rate,
        "opt_rate": best_history[-1][0],
        "opt_theta": opt_theta,
        "theory_ratio": theory_ratio,
        "ai_ratio": ai_ratio
    }, f, indent=2)

CELL 23: HARDWARE-SOFTWARE CO-DESIGN (CPS OPTIMIZER)
Generating Common Random Numbers (CRN) for 1000 trials...
Hardware noise frozen at p_V=0.12, p_H=0.04
CRN Generation Complete. Environment Frozen.

Evaluating Naive Baseline Decoder (w_V=1.0, w_H=1.0)...
 -> Baseline Logical Failure Rate: 0.0270

Starting CPS Evolutionary Engine...
GEN 01/8 | Best Fail Rate: 0.0180 | Weights: w_V=0.687, w_H=1.072
GEN 02/8 | Best Fail Rate: 0.0180 | Weights: w_V=0.687, w_H=1.072
GEN 03/8 | Best Fail Rate: 0.0180 | Weights: w_V=0.687, w_H=1.072
GEN 04/8 | Best Fail Rate: 0.0180 | Weights: w_V=0.687, w_H=1.072
GEN 05/8 | Best Fail Rate: 0.0180 | Weights: w_V=0.687, w_H=1.072
GEN 06/8 | Best Fail Rate: 0.0180 | Weights: w_V=0.687, w_H=1.072
GEN 07/8 | Best Fail Rate: 0.0180 | Weights: w_V=0.687, w_H=1.072
GEN 08/8 | Best Fail Rate: 0.0180 | Weights: w_V=0.687, w_H=1.072

CPS OPTIMIZATION COMPLETE IN 108.1s
NAIVE DECODER FAILURE RATE:  0.0270 (Uniform Weights)
CPS OPTIMIZED FAILURE RATE:  0.0180 (Learned 

In [43]:
import os

def generate_directory_report(root_path):
    report_lines = [f"Directory Report for: {root_path}\n" + "="*50 + "\n"]

    if not os.path.exists(root_path):
        print(f"Error: Path {root_path} does not exist.")
        return

    for root, dirs, files in os.walk(root_path):
        # Calculate depth for indentation
        level = root.replace(root_path, '').count(os.sep)
        indent = ' ' * 4 * level
        sub_indent = ' ' * 4 * (level + 1)

        # Add folder to report
        folder_name = os.path.basename(root) if level > 0 else root
        report_lines.append(f"{indent}[Folder] {folder_name}/")

        # Add files to report
        for f in sorted(files):
            # Skip the manifest file if it already exists to avoid recursion in the text
            if f == "directory_manifest.txt": continue
            report_lines.append(f"{sub_indent}- {f}")

    # Create the final string
    full_report = "\n".join(report_lines)

    # 1. Print to console
    print(full_report)

    # 2. Save to file
    manifest_path = os.path.join(root_path, "directory_manifest.txt")
    with open(manifest_path, "w") as f:
        f.write(full_report)

    print(f"\n[Success] Manifest saved to: {manifest_path}")

# Execute
target = "/content/shadowmap_cps_bootstrap"
generate_directory_report(target)

Directory Report for: /content/shadowmap_cps_bootstrap

[Folder] /content/shadowmap_cps_bootstrap/
    - cell0_environment_report.json
    [Folder] engine_runs/
        [Folder] true_topological_run_20260215T045456/
            - true_topological_results.json
        [Folder] surface_mwpm_run_20260215T020317/
            - evolution_log.json
        [Folder] cps_policy_optimization_20260215T051016/
            - policy_opt.json
        [Folder] surface_mc_run_20260215T020217/
            - evolution_log.json
        [Folder] surface_run_20260215T020216/
            - evolution_log.json
        [Folder] graph_run_20260215T020315/
            - evolution_log.json
        [Folder] run_20260215T020315/
            - evolution_log.json
        [Folder] graph_run_20260215T020216/
            - evolution_log.json
        [Folder] run_20260215T020216/
            - evolution_log.json
        [Folder] threshold_fixed_d_run_20260215T022004/
            - fixed_d_curves.json
            - monoton

files utility within Colab to trigger a direct browser download. This will zip your folder locally on the VM and then push it to your computer.

The "Zip and Download" Script
Run this cell to compress the folder and start the download:

In [47]:
import shutil
import os
from google.colab import files

# 1. Configuration
folder_to_zip = '/content/shadowmap_cps_bootstrap'
output_filename = 'shadowmap_bootstrap_full_20260215.zip'

# 2. Check if folder exists
if os.path.exists(folder_to_zip):
    print(f"Compressing {folder_to_zip}...")

    # Create the zip file in /content/
    shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)

    # 3. Trigger the browser download
    print(f"Starting download of {output_filename}...")
    files.download(output_filename)
else:
    print(f"Error: {folder_to_zip} not found.")

Compressing /content/shadowmap_cps_bootstrap...
Starting download of shadowmap_bootstrap_full_20260215.zip...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
# ==============================================================================
# CELL 24 — VALIDATION PACK (CI + paired McNemar + holdout generalization)
# Run BELOW Cell 23
# ==============================================================================

import os, json, glob, math
import numpy as np
import networkx as nx
from math import comb

# -----------------------------
# Utility: Wilson CI
# -----------------------------
def wilson_ci(k, n, z=1.96):
    if n == 0:
        return (0.0, 0.0)
    phat = k / n
    denom = 1.0 + (z*z)/n
    center = (phat + (z*z)/(2*n)) / denom
    half = (z * math.sqrt((phat*(1-phat) + (z*z)/(4*n)) / n)) / denom
    lo = max(0.0, center - half)
    hi = min(1.0, center + half)
    return (lo, hi)

# -----------------------------
# Utility: exact McNemar p-value (two-sided)
# -----------------------------
def mcnemar_exact_p(n01, n10):
    # Under H0, n01 ~ Bin(n01+n10, 0.5)
    n = n01 + n10
    if n == 0:
        return 1.0
    x = min(n01, n10)
    p_le_x = sum(comb(n, i) * (0.5 ** n) for i in range(x + 1))
    p_two_sided = min(1.0, 2.0 * p_le_x)
    return p_two_sided

# -----------------------------
# Load latest optimized policy (robust to kernel resets)
# -----------------------------
BASE_DIR = "/content/shadowmap_cps_bootstrap"
ENGINE_DIR = os.path.join(BASE_DIR, "engine_runs")
policy_paths = sorted(glob.glob(os.path.join(ENGINE_DIR, "cps_policy_optimization_*", "policy_opt.json")))

opt_theta = None
if policy_paths:
    latest_policy = policy_paths[-1]
    with open(latest_policy, "r") as f:
        saved = json.load(f)
    opt_theta = saved.get("opt_theta", None)
else:
    latest_policy = None

# Defaults from your printed run (used if variables not in memory)
d_default = 7
p_V_default = 0.12
p_H_default = 0.04
trials_default = 1000
seed_train_default = 42

# Prefer in-memory values if present
d = globals().get("d", d_default)
p_V = globals().get("p_V", p_V_default)
p_H = globals().get("p_H", p_H_default)
trials = globals().get("trials", trials_default)
seed_train = globals().get("seed_crn", seed_train_default)

# If opt_theta not found, fall back to the printed weights you pasted
if opt_theta is None:
    opt_theta = {"w_V": 0.687, "w_H": 1.072}

baseline_theta = {"w_V": 1.0, "w_H": 1.0}
theory_theta = {
    "w_V": math.log((1.0 - p_V) / p_V),
    "w_H": math.log((1.0 - p_H) / p_H),
}

# -----------------------------
# Re-implement the exact Cell 23 world + evaluation
# -----------------------------
def build_edge_list_and_probs(d_val, pV, pH):
    E = []
    probs = []

    # Vertical links
    for r in range(d_val - 1):
        for c in range(d_val - 1):
            E.append(("V", r, c)); probs.append(pV)

    # Horizontal links
    for r in range(d_val):
        for c in range(d_val - 2):
            E.append(("H", r, c)); probs.append(pH)

    # Boundary links (treated like "horizontal/boundary" in your cell)
    for r in range(d_val):
        E.append(("L", r)); probs.append(pH)
        E.append(("R", r)); probs.append(pH)

    return E, np.array(probs, dtype=float)

def generate_crn_dataset(d_val, pV, pH, ntrials, seed):
    rng = np.random.RandomState(seed)
    E, probs = build_edge_list_and_probs(d_val, pV, pH)
    dataset = []

    for _ in range(ntrials):
        err_mask = rng.rand(len(E)) < probs
        error_edges = {E[i] for i in range(len(E)) if err_mask[i]}

        syndrome = {}
        for edge in error_edges:
            tag = edge[0]
            if tag == "V":
                _, r, c = edge
                syndrome[(r, c)] = syndrome.get((r, c), 0) ^ 1
                syndrome[(r + 1, c)] = syndrome.get((r + 1, c), 0) ^ 1
            elif tag == "H":
                _, r, c = edge
                syndrome[(r, c)] = syndrome.get((r, c), 0) ^ 1
                syndrome[(r, c + 1)] = syndrome.get((r, c + 1), 0) ^ 1
            elif tag == "L":
                _, r = edge
                syndrome[(r, 0)] = syndrome.get((r, 0), 0) ^ 1
            elif tag == "R":
                _, r = edge
                syndrome[(r, d_val - 2)] = syndrome.get((r, d_val - 2), 0) ^ 1

        defects = [node for node, s in syndrome.items() if s == 1]
        dataset.append((error_edges, defects))

    return dataset

def get_weighted_dist(u, v, d_val, w_V, w_H):
    if u == "B" and v == "B":
        return 0.0
    if u == "B":
        u, v = v, u
    if v == "B":
        # distance to nearest boundary (counts boundary edge too)
        return min(u[1] + 1, d_val - 1 - u[1]) * w_H
    return abs(u[0] - v[0]) * w_V + abs(u[1] - v[1]) * w_H

def get_correction_edges(u, v, d_val):
    edges = []
    if u == "B" and v == "B":
        return edges
    if u == "B":
        u, v = v, u

    r, c = u
    if v == "B":
        # route to nearest boundary
        if (c + 1) <= (d_val - 1 - c):
            for col in range(c - 1, -1, -1):
                edges.append(("H", r, col))
            edges.append(("L", r))
        else:
            for col in range(c, d_val - 2):
                edges.append(("H", r, col))
            edges.append(("R", r))
        return edges

    r2, c2 = v
    # vertical steps
    step_r = 1 if r2 > r else -1
    for curr_r in range(r, r2, step_r):
        edges.append(("V", curr_r if step_r == 1 else curr_r - 1, c))
    # horizontal steps
    step_c = 1 if c2 > c else -1
    for curr_c in range(c, c2, step_c):
        edges.append(("H", r2, curr_c if step_c == 1 else curr_c - 1))
    return edges

def evaluate_policy_trace(theta, dataset, d_val):
    w_V = max(0.01, float(theta["w_V"]))
    w_H = max(0.01, float(theta["w_H"]))
    left_cut = {("L", r) for r in range(d_val)}

    fails = []
    for error_edges, defects in dataset:
        correction_edges = set()

        if defects:
            K = nx.Graph()
            for i, u in enumerate(defects):
                shadow = f"s_{i}"
                K.add_node(i); K.add_node(shadow)
                K.add_edge(i, shadow, weight=get_weighted_dist(u, "B", d_val, w_V, w_H))
                for j in range(i + 1, len(defects)):
                    v = defects[j]
                    K.add_edge(i, j, weight=get_weighted_dist(u, v, d_val, w_V, w_H))
                    K.add_edge(shadow, f"s_{j}", weight=0)

            matching = nx.algorithms.matching.min_weight_matching(K, weight="weight")

            for a, b in matching:
                if str(a).startswith("s_") and str(b).startswith("s_"):
                    continue

                node_a = "B" if str(a).startswith("s_") else defects[a]
                node_b = "B" if str(b).startswith("s_") else defects[b]

                for e in get_correction_edges(node_a, node_b, d_val):
                    if e in correction_edges:
                        correction_edges.remove(e)
                    else:
                        correction_edges.add(e)

        residual = error_edges.symmetric_difference(correction_edges)
        fail = (sum(1 for e in residual if e in left_cut) % 2) == 1
        fails.append(1 if fail else 0)

    k = sum(fails)
    n = len(fails)
    rate = k / n if n else 0.0
    return {
        "k": k, "n": n, "rate": rate,
        "ci95": wilson_ci(k, n),
        "fails": fails,
        "w_V": w_V, "w_H": w_H, "ratio": (w_V / w_H),
    }

# -----------------------------
# Build train + test datasets
# -----------------------------
train = generate_crn_dataset(d, p_V, p_H, trials, seed_train)
test  = generate_crn_dataset(d, p_V, p_H, trials, seed_train + 999)

# -----------------------------
# Evaluate
# -----------------------------
train_base = evaluate_policy_trace(baseline_theta, train, d)
train_opt  = evaluate_policy_trace(opt_theta, train, d)
train_th   = evaluate_policy_trace(theory_theta, train, d)

test_base  = evaluate_policy_trace(baseline_theta, test, d)
test_opt   = evaluate_policy_trace(opt_theta, test, d)
test_th    = evaluate_policy_trace(theory_theta, test, d)

# Paired McNemar (baseline vs opt on TRAIN)
n01 = sum(1 for b, o in zip(train_base["fails"], train_opt["fails"]) if b == 1 and o == 0)
n10 = sum(1 for b, o in zip(train_base["fails"], train_opt["fails"]) if b == 0 and o == 1)
p_mcnemar = mcnemar_exact_p(n01, n10)

def fmt(name, r):
    lo, hi = r["ci95"]
    return f"{name:>8}: {r['rate']:.4f}  [{lo:.4f}, {hi:.4f}]  ({r['k']}/{r['n']})  ratio={r['ratio']:.3f}"

print("="*80)
print("CELL 24 — VALIDATION RESULTS")
print("="*80)
print(f"Params: d={d}, p_V={p_V}, p_H={p_H}, trials={trials}")
if latest_policy:
    print("Loaded opt_theta from:", latest_policy)
print("-"*80)
print("TRAIN (seed =", seed_train, ")")
print(fmt("baseline", train_base))
print(fmt("opt",      train_opt))
print(fmt("theory",   train_th))
print("-"*80)
print("TEST  (seed =", seed_train + 999, ")")
print(fmt("baseline", test_base))
print(fmt("opt",      test_opt))
print(fmt("theory",   test_th))
print("-"*80)
print("Paired McNemar (baseline vs opt, TRAIN):")
print(f"  n01 (baseline fail, opt success) = {n01}")
print(f"  n10 (baseline success, opt fail) = {n10}")
print(f"  exact two-sided p-value          = {p_mcnemar:.6f}")
print("-"*80)
print("opt_theta:", opt_theta)
print("theory_theta:", theory_theta)
print("="*80)


CELL 24 — VALIDATION RESULTS
Params: d=7, p_V=0.12, p_H=0.04, trials=1000
Loaded opt_theta from: /content/shadowmap_cps_bootstrap/engine_runs/cps_policy_optimization_20260215T051016/policy_opt.json
--------------------------------------------------------------------------------
TRAIN (seed = 42 )
baseline: 0.0270  [0.0186, 0.0390]  (27/1000)  ratio=1.000
     opt: 0.0180  [0.0114, 0.0283]  (18/1000)  ratio=0.641
  theory: 0.0180  [0.0114, 0.0283]  (18/1000)  ratio=0.627
--------------------------------------------------------------------------------
TEST  (seed = 1041 )
baseline: 0.0350  [0.0253, 0.0483]  (35/1000)  ratio=1.000
     opt: 0.0180  [0.0114, 0.0283]  (18/1000)  ratio=0.641
  theory: 0.0180  [0.0114, 0.0283]  (18/1000)  ratio=0.627
--------------------------------------------------------------------------------
Paired McNemar (baseline vs opt, TRAIN):
  n01 (baseline fail, opt success) = 15
  n10 (baseline success, opt fail) = 6
  exact two-sided p-value          = 0.078354